In [1]:
'''
Based on SymSpell:

Originally written in C#:

// SymSpell: 1 million times faster through Symmetric Delete spelling correction algorithm
//
// The Symmetric Delete spelling correction algorithm reduces the complexity of edit candidate generation and dictionary lookup 
// for a given Damerau-Levenshtein distance. It is six orders of magnitude faster and language independent.
// Opposite to other algorithms only deletes are required, no transposes + replaces + inserts.
// Transposes + replaces + inserts of the input term are transformed into deletes of the dictionary term.
// Replaces and inserts are expensive and language dependent: e.g. Chinese has 70,000 Unicode Han characters!
//
// Copyright (C) 2015 Wolf Garbe
// Version: 3.0
// Author: Wolf Garbe <wolf.garbe@faroo.com>
// Maintainer: Wolf Garbe <wolf.garbe@faroo.com>
// URL: http://blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/
// Description: http://blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/
//
// License:
// This program is free software; you can redistribute it and/or modify
// it under the terms of the GNU Lesser General Public License, 
// version 3.0 (LGPL-3.0) as published by the Free Software Foundation.
// http://www.opensource.org/licenses/LGPL-3.0
//
// Usage: single word + Enter:  Display spelling suggestions
//        Enter without input:  Terminate the program
'''
pass

In [2]:
import re
import math # GD: needed to calculate logs below
from scipy.stats import poisson # GD: needed to calculate emission probability
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
max_edit_distance = 3
not_found_str = '<not found>'

#Initialization:

In [4]:
def get_deletes_list(w):
    '''given a word, derive strings with up to max_edit_distance characters deleted'''
    deletes = []
    queue = [w]
    for d in range(max_edit_distance):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes

In [5]:
def create_dictionary_entry(w, dictionary, longest_word_length):
    '''add word and its derived deletions to dictionary'''
    # check if word is already in dictionary
    # dictionary entries are in the form: (list of suggested corrections, frequency of word in corpus)

    new_real_word_added = False
    if w in dictionary:
        dictionary[w] = (dictionary[w][0], dictionary[w][1] + 1)  # increment count of word in corpus
    else:
        dictionary[w] = ([], 1)  
        longest_word_length = max(longest_word_length, len(w))
        
    if dictionary[w][1]==1:
        # first appearance of word in corpus
        # n.b. word may already be in dictionary as a derived word (deleting character from a real word)
        # but counter of frequency of word in corpus is not incremented in those cases)
        
        new_real_word_added = True
        deletes = get_deletes_list(w)
        
        for item in deletes:
            if item in dictionary:
                # add (correct) word to delete's suggested correction list if not already there
                if item not in dictionary[item][0]:
                    dictionary[item][0].append(w)
            else:
                dictionary[item] = ([w], 0)  # note frequency of word in corpus is not incremented
        
    return new_real_word_added, longest_word_length

In [6]:
def create_dictionary(fname):
    
    print "Creating dictionary..." 

    dictionary = dict() # GD: moved here to ensure that dictionary is re-initialized.
    longest_word_length = 0 # GD: moved here to ensure that it is re-initialized.
    start_prob = dict()
    transition_prob = dict()
    word_count = 0
    
    with open(fname) as file:    
        
        for line in file:
            
            for sentence in line.split('.'): # GD: added to ensure split at sentence level
                
                words = re.findall('[a-z]+', sentence.lower())  # separate by words by non-alphabetical characters      
                
                for w, word in enumerate(words):
                    
                    new_word, longest_word_length = \
                        create_dictionary_entry(word, dictionary, longest_word_length)
                    
                    if new_word:
                        word_count += 1
                        
                    # GD: added to calculate probabilities for Hidden Markov Model
                    if w == 0:

                        # Probability of a word being at the beginning of a sentence
                        if word in start_prob:
                            start_prob[word] += 1
                        else:
                            start_prob[word] = 1
                    else:
                        
                        # Probability of transitionining from one word to another
                        # dictionary format: {previous word: {word1 : P(word1|prevous word), word2 : P(word2|prevous word)}}
                        # Check that prior word is present - create if not
                        if words[w - 1] not in transition_prob:
                            transition_prob[words[w - 1]] = dict()
                            
                        # Check that current word is present - create if not
                        if word not in transition_prob[words[w - 1]]:
                            transition_prob[words[w - 1]][word] = 0
                            
                        # Update value
                        transition_prob[words[w - 1]][word] += 1
                              
    # GD: added to convert counts to log-probabilities (to avoid underflow)
    # Note: natural logarithm, not base-10
    
    total_start_words = float(sum(start_prob.values()))
    default_start_prob = math.log(1/total_start_words)
    start_prob.update({k: math.log(v/total_start_words) for k, v in start_prob.items()})
    
    default_transition_prob = math.log(1./word_count)    
    transition_prob.update({k: {k1: math.log(float(v1)/sum(v.values())) for k1, v1 in v.items()} \
                            for k, v in transition_prob.items()})

    print "Total unique words in corpus: %i" % word_count
    print "Total items in dictionary (corpus words and deletions): %i" % len(dictionary)
    print "  Edit distance for deletions: %i" % max_edit_distance
    print "  Length of longest word in corpus: %i" % longest_word_length
    print "Total unique words appearing at the start of a sentence: %i" % len(start_prob)
    print "Total unique word transitions: %i" % len(transition_prob)
        
    return dictionary, longest_word_length, start_prob, default_start_prob, \
            transition_prob, default_transition_prob

In [7]:
%%time
dictionary, longest_word_length, start_prob, default_start_prob, transition_prob, default_transition_prob = \
    create_dictionary("testdata/big.txt")
pass

Creating dictionary...
Total unique words in corpus: 29157
Total items in dictionary (corpus words and deletions): 2151998
  Edit distance for deletions: 3
  Length of longest word in corpus: 18
Total unique words appearing at the start of a sentence: 15297
Total unique word transitions: 27224
CPU times: user 35.4 s, sys: 906 ms, total: 36.3 s
Wall time: 37.2 s


<div class="alert alert-info">
<p><b>For testing:</b></p>
<p>
Can look up a specific entry in the dictionary below. <br>
shows (possible corrections, and frequency that entry itself is in corpus - 0 if not a real word) <br>
Note: will return key error if there are no corrections (because we are accessing dictionary directly here)
</p>
</div>

In [8]:
try:
    print dictionary["essential"]
except KeyError:
    print 'Not in dictionary'

(['essentially', 'essentials'], 92)


In [9]:
try:
    print dictionary["wack"]
except KeyError:
    print 'Not in dictionary'

(['wrack'], 0)


#Word-level correction:

In [10]:
def dameraulevenshtein(seq1, seq2):
    """Calculate the Damerau-Levenshtein distance between sequences.

    Source: http://mwh.geek.nz/2009/04/26/python-damerau-levenshtein-distance/
    
    This distance is the number of additions, deletions, substitutions,
    and transpositions needed to transform the first sequence into the
    second. Although generally used with strings, any sequences of
    comparable objects will work.

    Transpositions are exchanges of *consecutive* characters; all other
    operations are self-explanatory.

    This implementation is O(N*M) time and O(M) space, for N and M the
    lengths of the two sequences.

    >>> dameraulevenshtein('ba', 'abc')
    2
    >>> dameraulevenshtein('fee', 'deed')
    2

    It works with arbitrary sequences too:
    >>> dameraulevenshtein('abcd', ['b', 'a', 'c', 'd', 'e'])
    2
    """
    # codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    # Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1 matrix.
    # However, only the current and two previous rows are needed at once,
    # so we only store those.
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    for x in xrange(len(seq1)):
        # Python lists wrap around for negative indices, so put the
        # leftmost column at the *end* of the list. This matches with
        # the zero-indexed strings and saves extra calculation.
        twoago, oneago, thisrow = oneago, thisrow, [0] * len(seq2) + [x + 1]
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)
            # This block deals with transpositions
            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
    return thisrow[len(seq2) - 1]

In [11]:
def get_suggestions(string, dictionary, longest_word_length, silent=False, min_count=0):
    '''
    return list of suggested corrections for potentially incorrectly spelled word
    '''
    
    if (len(string) - longest_word_length) > max_edit_distance:
        if not silent:
            print "no items in dictionary within maximum edit distance"
        return []
    
    suggest_dict = {}
    
    queue = [string]
    q_dictionary = {}  # items other than string that we've checked
    
    while len(queue)>0:
        q_item = queue[0]  # pop
        # print "processing '%s'" % q_item
        queue = queue[1:]
        
        # process queue item
        if (q_item in dictionary) and (q_item not in suggest_dict):
            if (dictionary[q_item][1]>=min_count):# GD added to trim list for context checking
            # word is in dictionary, and is a word from the corpus, and not already in suggestion list
            # so add to suggestion dictionary, indexed by the word with value (frequency in corpus, edit distance)
            # note q_items that are not the input string are shorter than input string 
            # since only deletes are added (unless manual dictionary corrections are added)
                assert len(string)>=len(q_item)
                suggest_dict[q_item] = (dictionary[q_item][1], len(string) - len(q_item))

            ## the suggested corrections for q_item as stored in dictionary (whether or not
            ## q_item itself is a valid word or merely a delete) can be valid corrections
            for sc_item in dictionary[q_item][0]:
                if (sc_item not in suggest_dict):
                    # compute edit distance
                    # if len(sc_item)==len(q_item):
                    #    item_dist = len(string) - len(q_item)
                    # suggested items should always be longer (unless manual corrections are added)
                    assert len(sc_item)>len(q_item)
                    # q_items that are not input should be shorter than original string 
                    # (unless manual corrections added)
                    assert len(q_item)<=len(string)
                    if len(q_item)==len(string):
                        assert q_item==string
                        item_dist = len(sc_item) - len(q_item)
                    #elif len(q_item)==len(string):
                        # a suggestion could be longer or shorter than original string (bug in original FAROO?)
                        # if suggestion is from string's suggestion list, sc_item will be longer
                        # if suggestion is from a delete's suggestion list, sc_item may be shorter
                    #   item_dist = abs(len(sc_item) - len(q_item))
                    #else:
                    # check in original code, but probably not necessary because string has already checked
                    assert sc_item!=string

                    # calculate edit distance using, for example, Damerau-Levenshtein distance
                    item_dist = dameraulevenshtein(sc_item, string)

                    if item_dist<=max_edit_distance:
                        assert sc_item in dictionary  # should already be in dictionary if in suggestion list
                        if (dictionary[q_item][1]>=min_count):# GD added to trim list for context checking
                            suggest_dict[sc_item] = (dictionary[sc_item][1], item_dist)

        # now generate deletes (e.g. a substring of string or of a delete) from the queue item
        # as additional items to check -- add to end of queue
        assert len(string)>=len(q_item)
        if (len(string)-len(q_item))<max_edit_distance and len(q_item)>1:
            for c in range(len(q_item)): # character index        
                word_minus_c = q_item[:c] + q_item[c+1:]
                if word_minus_c not in q_dictionary:
                    queue.append(word_minus_c)
                    q_dictionary[word_minus_c] = None  # arbitrary value, just to identify we checked this
             
    # queue is now empty: convert suggestions in dictionary to list for output
    
    if not silent:
        print "number of possible corrections: %i" %len(suggest_dict)
        print "  edit distance for deletions: %i" % max_edit_distance
    
    # output option 1
    # sort results by ascending order of edit distance and descending order of frequency
    #     and return list of suggested corrections only:
    # return sorted(suggest_dict, key = lambda x: (suggest_dict[x][1], -suggest_dict[x][0]))

    # output option 2
    # return list of suggestions with (correction, (frequency in corpus, edit distance)):
    as_list = suggest_dict.items()
    return sorted(as_list, key = lambda (term, (freq, dist)): (dist, -freq))

    '''
    Option 1:
    get_suggestions("file")
    ['file', 'five', 'fire', 'fine', ...]
    
    Option 2:
    get_suggestions("file")
    [('file', (5, 0)),
     ('five', (67, 1)),
     ('fire', (54, 1)),
     ('fine', (17, 1))...]  
    '''

<div class="alert alert-info">
<p><b>For testing:</b></p>
<p>
Type in word to correct below, to test and get whole list of possible suggestions.
</p>
</div>

In [12]:
%%time
get_suggestions("mittens", dictionary, longest_word_length)

number of possible corrections: 142
  edit distance for deletions: 3
CPU times: user 12 ms, sys: 1.54 ms, total: 13.5 ms
Wall time: 12.2 ms


[('mitten', (0, 1)),
 ('mittes', (0, 1)),
 ('ittens', (0, 1)),
 ('matters', (136, 2)),
 ('bitten', (13, 2)),
 ('kitten', (7, 2)),
 ('listens', (2, 2)),
 ('battens', (1, 2)),
 ('smitten', (1, 2)),
 ('itten', (0, 2)),
 ('mites', (0, 2)),
 ('miten', (0, 2)),
 ('ittes', (0, 2)),
 ('mtten', (0, 2)),
 ('mttes', (0, 2)),
 ('ttens', (0, 2)),
 ('itens', (0, 2)),
 ('miens', (0, 2)),
 ('mittn', (0, 2)),
 ('mitte', (0, 2)),
 ('ittns', (0, 2)),
 ('mitts', (0, 2)),
 ('matter', (365, 3)),
 ('sitting', (269, 3)),
 ('minutes', (146, 3)),
 ('written', (117, 3)),
 ('miles', (110, 3)),
 ('citizens', (109, 3)),
 ('letters', (108, 3)),
 ('listen', (100, 3)),
 ('cities', (77, 3)),
 ('bitter', (47, 3)),
 ('masters', (37, 3)),
 ('intense', (34, 3)),
 ('witness', (33, 3)),
 ('attend', (29, 3)),
 ('mistress', (24, 3)),
 ('fitted', (23, 3)),
 ('mines', (22, 3)),
 ('fitting', (21, 3)),
 ('miners', (19, 3)),
 ('mitenka', (16, 3)),
 ('tens', (16, 3)),
 ('sisters', (16, 3)),
 ('intent', (13, 3)),
 ('mothers', (12, 3)

In [13]:
%%time
#benchmark timing
for i in range(1000):
    get_suggestions("acamodation", dictionary, longest_word_length, silent=True)

CPU times: user 1.85 s, sys: 10 ms, total: 1.86 s
Wall time: 1.86 s


In [14]:
%%time
#benchmark timing
for i in range(1000):
    get_suggestions("acomodation", dictionary, longest_word_length, silent=True)

CPU times: user 2.13 s, sys: 16.6 ms, total: 2.14 s
Wall time: 2.14 s


In [15]:
%%time
#benchmark timing
for i in range(1000):
    get_suggestions("hous", dictionary, longest_word_length, silent=True)

CPU times: user 40.2 s, sys: 193 ms, total: 40.4 s
Wall time: 40.8 s


In [16]:
# get best word
def best_word(s, dictionary, silent=False):
    try:
        return get_suggestions(s, dictionary, longest_word_length, silent)[0]
    except:
        return None

<div class="alert alert-info">
<p><b>For testing:</b></p>
<p>
Type in word to correct below, to test and get most suggested word.
</p>
</div>

In [17]:
best_word("hello", dictionary)

number of possible corrections: 349
  edit distance for deletions: 3


('hello', (1, 0))

In [18]:
def correct_document(fname, dictionary):
    with open(fname) as file:
        doc_word_count = 0
        corrected_word_count = 0
        unknown_word_count = 0
        print "Finding misspelled words in your document..." 
        
        for i, line in enumerate(file):
            doc_words = re.findall('[a-z]+', line.lower())  # separate by words by non-alphabetical characters      
            for doc_word in doc_words:
                doc_word_count += 1
                suggestion = best_word(doc_word, dictionary, silent=True)
                if suggestion is None:
                    print "In line %i, the word < %s > was not found (no suggested correction)" % (i, doc_word)
                    unknown_word_count += 1
                elif suggestion[0]!=doc_word:
                    print "In line %i, %s: suggested correction is < %s >" % (i, doc_word, suggestion[0])
                    corrected_word_count += 1
        
    print "-----"
    print "total words checked: %i" % doc_word_count
    print "total unknown words: %i" % unknown_word_count
    print "total potential errors found: %i" % corrected_word_count

    return

<div class="alert alert-info">
<p><b>For testing:</b></p>
<p>
Provide text file to correct, and give all best word suggestions (word level only) for errors.
</p>
</div>

In [19]:
correct_document("testdata/test.txt", dictionary)

Finding misspelled words in your document...
-----
total words checked: 27
total unknown words: 0
total potential errors found: 0


In [20]:
# from http://www.columbia.edu/acis/cria/rosenberg/sample/
correct_document("testdata/OCRsample.txt", dictionary)

Finding misspelled words in your document...
In line 3, taiths: suggested correction is < taits >
In line 11, the word < oonipiittee > was not found (no suggested correction)
In line 13, tj: suggested correction is < to >
In line 13, mnnff: suggested correction is < snuff >
In line 13, gjpt: suggested correction is < gpt >
In line 15, unuer: suggested correction is < under >
In line 20, mthiitt: suggested correction is < miitt >
In line 22, pythian: suggested correction is < ythian >
In line 28, debbs: suggested correction is < debts >
In line 29, nericans: suggested correction is < ericans >
In line 33, unorthodox: suggested correction is < orthodox >
In line 33, nenance: suggested correction is < penance >
In line 38, williaij: suggested correction is < william >
In line 40, fcsf: suggested correction is < fcs >
In line 42, unorthodoxy: suggested correction is < orthodox >
In line 42, thpt: suggested correction is < that >
In line 42, the word < senbrnrgs > was not found (no suggeste

#Context-level correction:

**Model setup:**  
Each sentence is modeled as a hidden Markov model. Prior probabilities (for first word in the sentence) and transition probabilities (for all subsequent words) are calculated when generating the main dictionary, using the same corpus. Emission probabilities are generated on the fly by parameterizing a Poisson distribution with the edit distance. The state space of possible corrections is based on the suggested words from the word-level correction.  
All probabilities are stored in log-space to avoid underflow. Pre-defined minimum values are used for words that are not present in the dictionary and/or probability tables.

In [22]:
def get_emission_prob(edit_dist, poisson_lambda=0.01):
    
    # Poisson(k, l), where k = edit distance and l=0.01
    # TODO - validate lambda parameter (taken from Verena's code)
    
    return math.log(poisson.pmf(edit_dist, poisson_lambda))

In [23]:
def get_start_prob(word, start_prob, default_start_prob):
    try:
        return start_prob[word]
    except KeyError:
        return default_start_prob

In [24]:
def get_transition_prob(cur_word, prev_word, transition_prob, default_transition_prob):
    try:
        return transition_prob[prev_word][cur_word]
    except KeyError:
        return default_transition_prob

In [25]:
def get_belief(prev_word, prev_belief):
    try:
        return prev_belief[prev_word]
    except KeyError:
        return math.log(math.exp(min(prev_belief.values()))/2.) # TODO - confirm default value

In [73]:
# Modified from AM207 lecture notes
def viterbi(words, dictionary, longest_word_length, start_prob, default_start_prob, \
                transition_prob, default_transition_prob, \
                num_word_suggestions=5000):
    
    V = [{}]
    path = {}
    path_context = []
#     path_word = []
    
    # FOR TESTING - DELETE EVENTUALLY
    if type(words) != list:
        words = re.findall('[a-z]+', words.lower())  # separate by words by non-alphabetical characters
        
    # Character level correction
    corrections = get_suggestions(words[0], dictionary, longest_word_length, \
                                  silent=True, min_count=1)

    # To ensure Viterbi can keep running
    if len(corrections) == 0:
        corrections = [(words[0], (1, 0))]
#         path_word.append(not_found_str)
    else:    
        if len(corrections) > num_word_suggestions:
            corrections = corrections[0:num_word_suggestions]
#         if len(corrections) > 0:
#             path_word.append(corrections[0][0])  # string of most frequent word tuple
        
    # Initialize base cases (t == 0)
    for sug_word in corrections:
        
        # compute the value for all possible starting states
        V[0][sug_word[0]] = math.exp(get_start_prob(sug_word[0], start_prob, default_start_prob) \
                                     + get_emission_prob(sug_word[1][1]))
        
        # remember all the different paths (here its only one state so far)
        path[sug_word[0]] = [sug_word[0]]
 
#     print path
    
    # normalize for numerical stability
    path_temp_sum = sum(V[0].values())
    V[0].update({k: math.log(v/path_temp_sum) for k, v in V[0].items()})
    prev_corrections = [i[0] for i in corrections]
    
    if len(words) == 1:
        path_context = [max(V[0], key=lambda i: V[0][i])]
        return [], path_context
#         return path_word, path_context

    # Run Viterbi for t > 0
    for t in range(1, len(words)):

        V.append({})
        new_path = {}
        
        # Character level correction
        corrections = get_suggestions(words[t], dictionary, longest_word_length, \
                        silent=True, min_count=1)
        
        # To ensure Viterbi can keep running
        if len(corrections) == 0:
            corrections = [(words[t], (1, 0))]
#             path_word.append(not_found_str)
        else:
            if len(corrections) > num_word_suggestions:
                corrections = corrections[0:num_word_suggestions]
#             if len(corrections) > 0:
#                 path_word.append(corrections[0][0])  # string of most frequent word tuple

        for sug_word in corrections:
        
            sug_word_emission_prob = get_emission_prob(sug_word[1][1])
            
            # compute the values coming from all possible previous states, only keep the maximum
            (prob, word) = max((get_belief(prev_word, V[t-1]) \
                            + get_transition_prob(sug_word[0], prev_word, transition_prob, default_transition_prob) \
                            + sug_word_emission_prob, prev_word) for prev_word in prev_corrections)1            
            # save the maximum value for each state
            V[t][sug_word[0]] = math.exp(prob)
            # remember the path we came from to get this maximum value
            new_path[sug_word[0]] = path[word] + [sug_word[0]]
            
        # normalize for numerical stability
        path_temp_sum = sum(V[t].values())
        V[t].update({k: math.log(v/path_temp_sum) for k, v in V[t].items()})
        prev_corrections = [i[0] for i in corrections]
 
        # Don't need to remember the old paths
        path = new_path
     
    (prob, word) = max((V[t][sug_word[0]], sug_word[0]) for sug_word in corrections)
    path_context = path[word]
    
#     assert len(path_word) == len(path_context)

#     return path_word, path_context
    return [], path_context

In [74]:
def correct_document_context(fname, dictionary, longest_word_length, start_prob, default_start_prob, \
                             transition_prob, default_transition_prob, num_word_suggestions=5000):
    
    doc_word_count = 0
    unknown_word_count = 0
    corrected_word_count = 0
    mismatches = 0
    
    with open(fname) as file:
        
        for i, line in enumerate(file):
            
            for sentence in line.split('.'):
                
                words = re.findall('[a-z]+', sentence.lower())  # separate by words by non-alphabetical characters
                doc_word_count += len(words)
                
                if len(words) > 0:
                
                    suggestion_w, suggestion_c = viterbi(words, dictionary, longest_word_length, \
                                                start_prob, default_start_prob, \
                                                transition_prob, default_transition_prob)

                    # Display sentences where errors have been identified
                    if (words != suggestion_w) or (words != suggestion_c):
                        
                        # Check for unknown words
                        unknown_word_count += sum([w==not_found_str for w in suggestion_w])
                        
                        # Most users will expect to see 1-indexing.
                        print '\nErrors found in line %i. \nOriginal sentence: %s' % (i+1, " ".join(words))

                        # Word-checker and context-checker output match
                        if suggestion_w == suggestion_c:
                            print 'Word & context-level correction: %s' % (" ".join(suggestion_w))
                            corrected_word_count += sum([words[j]!=suggestion_w[j] for j in range(len(words))])
                        
                        # Word-checker and context-checker output don't match
                        else:
                            print 'Word-level correction: %s' % (" ".join(suggestion_w))
                            print 'Context-level correction: %s' % (" ".join(suggestion_c))
                            corrected_word_count += \
                                sum([(words[j]!=suggestion_w[j]) or (words[j]!=suggestion_c[j]) for j in range(len(words))])
                            mismatches += sum([suggestion_w[j] != suggestion_c[j] for j in range(len(words))])
  
    print "-----"
    print "total words checked: %i" % doc_word_count
    print "total unknown words: %i" % unknown_word_count
    print "total potential errors found: %i" % corrected_word_count
    print "total mismatches (word-level vs. context-level): %i" % mismatches

<div class="alert alert-info">
<p><b>For testing:</b></p>
<p>
Provide string to correct, and give all best word suggestions (word level & context level).
</p>
</div>

In [75]:
%%time
sentence = "this is ax test"
word_check, context_check = viterbi(sentence, \
            dictionary, longest_word_length, start_prob, default_start_prob, transition_prob, default_transition_prob)
print 'Original sentence: ', sentence
print 'Word-level check: ', " ".join(word_check)
print 'Context-level check: ', " ".join(context_check)

('is', (9774, 0)) (-2.530211929901725, 'this')
('in', (22050, 1)) (-10.002612345224286, 'this')
('it', (10681, 1)) (-11.449531328160612, 'this')
('his', (10034, 1)) (-11.737213400612394, 'this')
('as', (8064, 1)) (-9.945453931384337, 'this')
('i', (7687, 1)) (-10.350919039492503, 'this')
('s', (5636, 1)) (-14.906151584287816, 'this')
('if', (2373, 1)) (-12.835825689280501, 'this')
('its', (1635, 1)) (-14.906151584287816, 'this')
('us', (684, 1)) (-14.906151584287816, 'this')
('ii', (77, 1)) (-14.906151584287816, 'this')
('iv', (55, 1)) (-14.906151584287816, 'this')
('isn', (52, 1)) (-12.142678508720557, 'this')
('ix', (28, 1)) (-14.906151584287816, 'this')
('il', (18, 1)) (-14.906151584287816, 'this')
('vs', (9, 1)) (-14.906151584287816, 'this')
('si', (5, 1)) (-14.906151584287816, 'this')
('tis', (5, 1)) (-14.906151584287816, 'this')
('isa', (4, 1)) (-14.906151584287816, 'this')
('es', (3, 1)) (-14.906151584287816, 'this')
('irs', (3, 1)) (-14.906151584287816, 'this')
('os', (3, 1)) (

In [56]:
test = [('is', (9774, 0)), ('in', (22050, 1)), ('it', (10681, 1)), ('his', (10034, 1)), ('as', (8064, 1)), ('i', (7687, 1)), ('s', (5636, 1)), ('if', (2373, 1)), ('its', (1635, 1)), ('us', (684, 1)), ('ii', (77, 1)), ('iv', (55, 1)), ('isn', (52, 1)), ('ix', (28, 1)), ('il', (18, 1)), ('vs', (9, 1)), ('si', (5, 1)), ('tis', (5, 1)), ('isa', (4, 1)), ('es', (3, 1)), ('irs', (3, 1)), ('os', (3, 1)), ('ms', (2, 1)), ('iss', (2, 1)), ('iso', (2, 1)), ('dis', (1, 1)), ('vis', (1, 1)), ('zis', (1, 1)), ('ist', (1, 1)), ('ism', (1, 1)), ('lis', (1, 1)), ('id', (1, 1)), ('was', (11410, 2)), ('him', (5230, 2)), ('this', (4063, 2)), ('so', (3017, 2)), ('did', (1875, 2)), ('has', (1603, 2)), ('yes', (688, 2)), ('use', (320, 2)), ('fig', (320, 2)), ('ask', (251, 2)), ('wish', (243, 2)), ('rise', (240, 2)), ('air', (228, 2)), ('sir', (177, 2)), ('six', (176, 2)), ('pus', (155, 2)), ('st', (147, 2)), ('lips', (146, 2)), ('ill', (138, 2)), ('aid', (113, 2)), ('miss', (112, 2)), ('aim', (108, 2)), ('iii', (91, 2)), ('sit', (89, 2)), ('lie', (89, 2)), ('die', (86, 2)), ('lies', (86, 2)), ('big', (82, 2)), ('risk', (75, 2)), ('lit', (74, 2)), ('bit', (63, 2)), ('mrs', (59, 2)), ('lip', (56, 2)), ('fit', (52, 2)), ('list', (48, 2)), ('kiss', (47, 2)), ('hip', (47, 2)), ('rid', (42, 2)), ('mist', (41, 2)), ('oil', (39, 2)), ('win', (38, 2)), ('axis', (37, 2)), ('vi', (37, 2)), ('vii', (34, 2)), ('lise', (30, 2)), ('est', (29, 2)), ('ice', (29, 2)), ('xi', (28, 2)), ('xii', (28, 2)), ('xiv', (28, 2)), ('hid', (28, 2)), ('fix', (27, 2)), ('inn', (26, 2)), ('ties', (24, 2)), ('dim', (24, 2)), ('aims', (24, 2)), ('wise', (24, 2)), ('sin', (23, 2)), ('pas', (23, 2)), ('fish', (21, 2)), ('hit', (21, 2)), ('les', (18, 2)), ('pin', (18, 2)), ('xix', (17, 2)), ('dies', (17, 2)), ('tips', (16, 2)), ('gas', (16, 2)), ('ribs', (16, 2)), ('wit', (15, 2)), ('pit', (15, 2)), ('tie', (15, 2)), ('fits', (14, 2)), ('lid', (14, 2)), ('ink', (14, 2)), ('dish', (13, 2)), ('figs', (12, 2)), ('pips', (12, 2)), ('disk', (10, 2)), ('sins', (10, 2)), ('des', (10, 2)), ('dip', (9, 2)), ('otis', (9, 2)), ('pig', (9, 2)), ('lids', (8, 2)), ('fist', (8, 2)), ('bits', (8, 2)), ('rib', (8, 2)), ('tip', (8, 2)), ('tit', (7, 2)), ('oris', (7, 2)), ('bid', (7, 2)), ('ti', (7, 2)), ('wits', (7, 2)), ('tin', (7, 2)), ('disc', (6, 2)), ('fir', (6, 2)), ('mid', (6, 2)), ('ein', (6, 2)), ('sits', (6, 2)), ('wisp', (6, 2)), ('din', (5, 2)), ('dig', (5, 2)), ('wig', (5, 2)), ('ash', (5, 2)), ('pins', (5, 2)), ('mais', (5, 2)), ('gin', (5, 2)), ('kit', (5, 2)), ('pie', (5, 2)), ('mix', (4, 2)), ('zip', (4, 2)), ('sa', (4, 2)), ('pits', (4, 2)), ('rim', (4, 2)), ('suis', (4, 2)), ('gig', (3, 2)), ('kin', (3, 2)), ('ices', (3, 2)), ('mi', (3, 2)), ('wins', (3, 2)), ('se', (3, 2)), ('ills', (3, 2)), ('airs', (3, 2)), ('ivy', (3, 2)), ('hips', (3, 2)), ('inns', (3, 2)), ('hiss', (3, 2)), ('gist', (3, 2)), ('firs', (3, 2)), ('ing', (3, 2)), ('isle', (3, 2)), ('via', (3, 2)), ('vie', (3, 2)), ('tic', (3, 2)), ('esq', (3, 2)), ('iris', (3, 2)), ('los', (3, 2)), ('ai', (3, 2)), ('sic', (2, 2)), ('wigs', (2, 2)), ('pigs', (2, 2)), ('ida', (2, 2)), ('jim', (2, 2)), ('aids', (2, 2)), ('ass', (2, 2)), ('inst', (2, 2)), ('aix', (2, 2)), ('mio', (2, 2)), ('vise', (2, 2)), ('bin', (2, 2)), ('kits', (2, 2)), ('ses', (2, 2)), ('til', (2, 2)), ('sip', (2, 2)), ('di', (2, 2)), ('bias', (2, 2)), ('nip', (2, 2)), ('kid', (2, 2)), ('ici', (2, 2)), ('pes', (2, 2)), ('rio', (2, 2)), ('icy', (2, 2)), ('hi', (2, 2)), ('bids', (2, 2)), ('ni', (2, 2)), ('grs', (2, 2)), ('fas', (1, 2)), ('li', (1, 2)), ('dio', (1, 2)), ('lisp', (1, 2)), ('dix', (1, 2)), ('lisa', (1, 2)), ('cms', (1, 2)), ('wie', (1, 2)), ('oils', (1, 2)), ('git', (1, 2)), ('las', (1, 2)), ('sq', (1, 2)), ('sn', (1, 2)), ('sg', (1, 2)), ('sd', (1, 2)), ('tss', (1, 2)), ('das', (1, 2)), ('ri', (1, 2)), ('asp', (1, 2)), ('wid', (1, 2)), ('wiz', (1, 2)), ('digs', (1, 2)), ('ras', (1, 2)), ('vish', (1, 2)), ('ces', (1, 2)), ('hise', (1, 2)), ('ire', (1, 2)), ('imp', (1, 2)), ('hits', (1, 2)), ('ain', (1, 2)), ('rip', (1, 2)), ('vos', (1, 2)), ('ion', (1, 2)), ('min', (1, 2)), ('tim', (1, 2)), ('vif', (1, 2)), ('pies', (1, 2)), ('esp', (1, 2)), ('viz', (1, 2)), ('nos', (1, 2)), ('obs', (1, 2)), ('vas', (1, 2)), ('bus', (1, 2)), ('fils', (1, 2)), ('ind', (1, 2)), ('cos', (1, 2)), ('ich', (1, 2)), ('with', (9740, 3)), ('she', (3946, 3)), ('said', (3464, 3)), ('into', (2124, 3)), ('will', (1577, 3)), ('time', (1529, 3)), ('first', (1177, 3)), ('see', (1101, 3)), ('like', (1080, 3)), ('must', (955, 3)), ('eyes', (939, 3)), ('most', (908, 3)), ('life', (878, 3)), ('also', (778, 3)), ('just', (767, 3)), ('say', (755, 3)), ('saw', (599, 3)), ('skin', (579, 3)), ('last', (565, 3)), ('give', (523, 3)), ('side', (511, 3)), ('days', (453, 3)), ('case', (438, 3)), ('sat', (403, 3)), ('less', (367, 3)), ('wife', (367, 3)), ('does', (358, 3)), ('son', (343, 3)), ('mind', (341, 3)), ('set', (325, 3)), ('pain', (303, 3)), ('find', (294, 3)), ('boris', (293, 3)), ('high', (290, 3)), ('west', (286, 3)), ('five', (279, 3)), ('used', (276, 3)), ('fire', (274, 3)), ('best', (268, 3)), ('arms', (249, 3)), ('line', (248, 3)), ('rose', (243, 3)), ('king', (238, 3)), ('times', (236, 3)), ('hair', (234, 3)), ('laws', (233, 3)), ('news', (224, 3)), ('lost', (224, 3)), ('past', (223, 3)), ('limb', (217, 3)), ('till', (216, 3)), ('thus', (212, 3)), ('rest', (209, 3)), ('kind', (202, 3)), ('else', (201, 3)), ('laid', (186, 3)), ('size', (184, 3)), ('city', (180, 3)), ('view', (179, 3)), ('fine', (175, 3)), ('sides', (172, 3)), ('thin', (166, 3)), ('girl', (166, 3)), ('pass', (154, 3)), ('says', (146, 3)), ('loss', (143, 3)), ('idea', (141, 3)), ('firm', (141, 3)), ('veins', (134, 3)), ('lines', (133, 3)), ('east', (130, 3)), ('live', (128, 3)), ('wait', (127, 3)), ('easy', (122, 3)), ('legs', (118, 3)), ('post', (117, 3)), ('vein', (116, 3)), ('guns', (112, 3)), ('miles', (110, 3)), ('main', (109, 3)), ('hill', (107, 3)), ('paid', (106, 3)), ('wide', (105, 3)), ('ends', (103, 3)), ('nose', (103, 3)), ('issue', (101, 3)), ('mass', (101, 3)), ('bill', (99, 3)), ('sign', (99, 3)), ('signs', (98, 3)), ('soil', (93, 3)), ('hills', (93, 3)), ('rich', (92, 3)), ('ohio', (92, 3)), ('sad', (91, 3)), ('acts', (89, 3)), ('gives', (88, 3)), ('rays', (87, 3)), ('maid', (87, 3)), ('amid', (84, 3)), ('died', (83, 3)), ('sun', (82, 3)), ('sea', (82, 3)), ('visit', (81, 3)), ('paris', (80, 3)), ('iron', (78, 3)), ('ones', (77, 3)), ('pity', (75, 3)), ('sac', (73, 3)), ('ship', (73, 3)), ('girls', (71, 3)), ('wrist', (68, 3)), ('sky', (68, 3)), ('cyst', (67, 3)), ('wars', (67, 3)), ('limbs', (67, 3)), ('didn', (67, 3)), ('cost', (65, 3)), ('nine', (64, 3)), ('ships', (64, 3)), ('views', (64, 3)), ('moist', (63, 3)), ('join', (62, 3)), ('mine', (62, 3)), ('acid', (61, 3)), ('wine', (61, 3)), ('goes', (60, 3)), ('nail', (60, 3)), ('lives', (60, 3)), ('vice', (58, 3)), ('ears', (58, 3)), ('ideas', (57, 3)), ('kill', (57, 3)), ('tsar', (56, 3)), ('pipe', (56, 3)), ('sinus', (56, 3)), ('ring', (55, 3)), ('busy', (55, 3)), ('fair', (55, 3)), ('evil', (55, 3)), ('ride', (54, 3)), ('cast', (54, 3)), ('base', (53, 3)), ('sum', (53, 3)), ('sick', (53, 3)), ('test', (53, 3)), ('nice', (53, 3)), ('ways', (52, 3)), ('raise', (52, 3)), ('toes', (51, 3)), ('lose', (51, 3)), ('mild', (50, 3)), ('midst', (50, 3)), ('silk', (50, 3)), ('task', (49, 3)), ('wind', (49, 3)), ('vast', (48, 3)), ('ruin', (48, 3)), ('aide', (48, 3)), ('irish', (48, 3)), ('louis', (44, 3)), ('gain', (44, 3)), ('ease', (44, 3)), ('exist', (44, 3)), ('sons', (43, 3)), ('icon', (43, 3)), ('pains', (42, 3)), ('sobs', (41, 3)), ('fill', (41, 3)), ('tail', (40, 3)), ('tied', (40, 3)), ('pair', (40, 3)), ('hide', (40, 3)), ('fail', (40, 3)), ('basis', (40, 3)), ('whip', (40, 3)), ('rain', (39, 3)), ('mills', (39, 3)), ('viii', (39, 3)), ('dust', (39, 3)), ('fast', (39, 3)), ('sigh', (38, 3)), ('fires', (38, 3)), ('dogs', (38, 3)), ('lads', (38, 3)), ('noise', (38, 3)), ('seas', (37, 3)), ('sing', (36, 3)), ('bird', (36, 3)), ('whigs', (36, 3)), ('wild', (35, 3)), ('stir', (35, 3)), ('ours', (35, 3)), ('rush', (35, 3)), ('bills', (34, 3)), ('xvi', (34, 3)), ('ilya', (33, 3)), ('vous', (33, 3)), ('tide', (33, 3)), ('mile', (32, 3)), ('sire', (32, 3)), ('wing', (32, 3)), ('virus', (32, 3)), ('lift', (32, 3)), ('site', (32, 3)), ('nails', (32, 3)), ('cries', (31, 3)), ('hers', (30, 3)), ('chin', (30, 3)), ('risen', (30, 3)), ('vain', (30, 3)), ('maids', (30, 3)), ('rosy', (30, 3)), ('milk', (29, 3)), ('icons', (29, 3)), ('kinds', (29, 3)), ('arise', (28, 3)), ('rows', (28, 3)), ('vols', (27, 3)), ('runs', (27, 3)), ('minds', (27, 3)), ('kings', (27, 3)), ('pink', (27, 3)), ('sees', (26, 3)), ('lest', (25, 3)), ('dose', (25, 3)), ('suit', (25, 3)), ('rico', (24, 3)), ('slip', (24, 3)), ('rises', (24, 3)), ('bier', (24, 3)), ('bees', (24, 3)), ('sub', (23, 3)), ('lime', (23, 3)), ('wire', (23, 3)), ('rice', (23, 3)), ('finds', (23, 3)), ('host', (23, 3)), ('bind', (22, 3)), ('bite', (22, 3)), ('coin', (22, 3)), ('gets', (22, 3)), ('mines', (22, 3)), ('xiii', (22, 3)), ('file', (22, 3)), ('void', (21, 3)), ('hive', (21, 3)), ('shy', (21, 3)), ('asks', (21, 3)), ('pose', (21, 3)), ('boys', (21, 3)), ('reins', (20, 3)), ('pick', (20, 3)), ('iowa', (20, 3)), ('wash', (20, 3)), ('san', (20, 3)), ('gains', (20, 3)), ('fails', (20, 3)), ('tiny', (20, 3)), ('sums', (20, 3)), ('idle', (19, 3)), ('vive', (19, 3)), ('caps', (19, 3)), ('sob', (19, 3)), ('push', (19, 3)), ('mill', (18, 3)), ('piti', (18, 3)), ('exit', (18, 3)), ('jaws', (18, 3)), ('bass', (18, 3)), ('davis', (18, 3)), ('xvii', (18, 3)), ('evils', (18, 3)), ('whig', (18, 3)), ('kiev', (18, 3)), ('aides', (17, 3)), ('sill', (17, 3)), ('boil', (17, 3)), ('waist', (17, 3)), ('boils', (17, 3)), ('rings', (17, 3)), ('arts', (17, 3)), ('jest', (17, 3)), ('vile', (16, 3)), ('pipes', (16, 3)), ('penis', (16, 3)), ('ages', (16, 3)), ('tens', (16, 3)), ('sly', (16, 3)), ('wives', (16, 3)), ('veil', (16, 3)), ('hairs', (15, 3)), ('inch', (15, 3)), ('fees', (15, 3)), ('oats', (15, 3)), ('writ', (15, 3)), ('pitt', (15, 3)), ('joys', (15, 3)), ('birds', (15, 3)), ('gems', (15, 3)), ('dirt', (14, 3)), ('knit', (14, 3)), ('akin', (14, 3)), ('spy', (14, 3)), ('dine', (14, 3)), ('wasn', (14, 3)), ('toil', (14, 3)), ('risks', (14, 3)), ('fiske', (14, 3)), ('quit', (14, 3)), ('erie', (14, 3)), ('bags', (14, 3)), ('ross', (14, 3)), ('twist', (14, 3)), ('sink', (13, 3)), ('slim', (13, 3)), ('grim', (13, 3)), ('links', (13, 3)), ('lint', (13, 3)), ('grip', (13, 3)), ('hint', (13, 3)), ('wills', (13, 3)), ('xxi', (13, 3)), ('units', (13, 3)), ('tipsy', (13, 3)), ('keys', (13, 3)), ('diet', (13, 3)), ('beds', (13, 3)), ('hints', (12, 3)), ('lens', (12, 3)), ('bast', (12, 3)), ('fills', (12, 3)), ('bliss', (12, 3)), ('ruins', (12, 3)), ('link', (12, 3)), ('trip', (12, 3)), ('dieu', (12, 3)), ('mask', (12, 3)), ('riot', (11, 3)), ('writs', (11, 3)), ('rein', (11, 3)), ('sex', (11, 3)), ('tint', (11, 3)), ('cash', (11, 3)), ('fuss', (11, 3)), ('sets', (11, 3)), ('desk', (11, 3)), ('wink', (11, 3)), ('wens', (11, 3)), ('gift', (11, 3)), ('rail', (11, 3)), ('hats', (10, 3)), ('xxii', (10, 3)), ('diam', (10, 3)), ('unit', (10, 3)), ('brisk', (10, 3)), ('heir', (10, 3)), ('bites', (10, 3)), ('mess', (10, 3)), ('enns', (10, 3)), ('foes', (10, 3)), ('furs', (10, 3)), ('dusk', (10, 3)), ('wings', (10, 3)), ('noisy', (10, 3)), ('misty', (10, 3)), ('roi', (9, 3)), ('fists', (9, 3)), ('adds', (9, 3)), ('hasn', (9, 3)), ('jail', (9, 3)), ('pine', (9, 3)), ('spies', (9, 3)), ('anus', (9, 3)), ('lists', (9, 3)), ('dash', (9, 3)), ('rags', (9, 3)), ('sites', (9, 3)), ('tons', (9, 3)), ('mail', (9, 3)), ('wail', (9, 3)), ('uses', (9, 3)), ('hind', (9, 3)), ('pile', (9, 3)), ('jews', (9, 3)), ('ivan', (9, 3)), ('fili', (9, 3)), ('qui', (8, 3)), ('suits', (8, 3)), ('limp', (8, 3)), ('xxiv', (8, 3)), ('guise', (8, 3)), ('eats', (8, 3)), ('nous', (8, 3)), ('whips', (8, 3)), ('pairs', (8, 3)), ('eggs', (8, 3)), ('lots', (8, 3)), ('sail', (8, 3)), ('bush', (8, 3)), ('gait', (8, 3)), ('bars', (8, 3)), ('huts', (8, 3)), ('lash', (8, 3)), ('flies', (8, 3)), ('hast', (8, 3)), ('likes', (8, 3)), ('heirs', (8, 3)), ('bows', (8, 3)), ('film', (8, 3)), ('user', (8, 3)), ('grin', (8, 3)), ('winds', (8, 3)), ('gums', (8, 3)), ('logs', (8, 3)), ('files', (8, 3)), ('gilt', (8, 3)), ('maps', (7, 3)), ('cars', (7, 3)), ('lieu', (7, 3)), ('rails', (7, 3)), ('pads', (7, 3)), ('rash', (7, 3)), ('owns', (7, 3)), ('raid', (7, 3)), ('nest', (7, 3)), ('lets', (7, 3)), ('wiser', (7, 3)), ('tubs', (7, 3)), ('liar', (6, 3)), ('amiss', (6, 3)), ('arid', (6, 3)), ('riots', (6, 3)), ('kick', (6, 3)), ('pike', (6, 3)), ('slit', (6, 3)), ('nuts', (6, 3)), ('pint', (6, 3)), ('vaso', (6, 3)), ('cutis', (6, 3)), ('whim', (6, 3)), ('pills', (6, 3)), ('lain', (6, 3)), ('omit', (6, 3)), ('dans', (6, 3)), ('sam', (6, 3)), ('diaz', (6, 3)), ('rides', (6, 3)), ('moss', (6, 3)), ('cubs', (6, 3)), ('debs', (6, 3)), ('bile', (6, 3)), ('ripe', (6, 3)), ('fuse', (6, 3)), ('swiss', (6, 3)), ('swim', (6, 3)), ('crisp', (6, 3)), ('hush', (6, 3)), ('sighs', (6, 3)), ('piles', (6, 3)), ('buds', (6, 3)), ('gifts', (6, 3)), ('item', (5, 3)), ('alms', (5, 3)), ('nigh', (5, 3)), ('axes', (5, 3)), ('coins', (5, 3)), ('chip', (5, 3)), ('rods', (5, 3)), ('tries', (5, 3)), ('brim', (5, 3)), ('pens', (5, 3)), ('pegs', (5, 3)), ('aime', (5, 3)), ('wipe', (5, 3)), ('lewis', (5, 3)), ('tier', (5, 3)), ('hire', (5, 3)), ('fait', (5, 3)), ('cuts', (5, 3)), ('coil', (5, 3)), ('boss', (5, 3)), ('trim', (5, 3)), ('ants', (5, 3)), ('slips', (5, 3)), ('vest', (5, 3)), ('lust', (5, 3)), ('mobs', (5, 3)), ('clip', (5, 3)), ('hail', (5, 3)), ('plus', (5, 3)), ('lied', (5, 3)), ('pays', (5, 3)), ('mails', (5, 3)), ('zinc', (4, 3)), ('tidy', (4, 3)), ('elias', (4, 3)), ('toss', (4, 3)), ('voir', (4, 3)), ('sans', (4, 3)), ('toils', (4, 3)), ('sue', (4, 3)), ('asia', (4, 3)), ('twins', (4, 3)), ('items', (4, 3)), ('oily', (4, 3)), ('pints', (4, 3)), ('chit', (4, 3)), ('ilio', (4, 3)), ('fines', (4, 3)), ('tails', (4, 3)), ('igni', (4, 3)), ('skins', (4, 3)), ('wines', (4, 3)), ('dries', (4, 3)), ('tres', (4, 3)), ('sings', (4, 3)), ('alas', (4, 3)), ('sait', (4, 3)), ('lui', (4, 3)), ('rite', (4, 3)), ('apes', (4, 3)), ('jets', (4, 3)), ('ruts', (4, 3)), ('twin', (4, 3)), ('edit', (4, 3)), ('dams', (4, 3)), ('sow', (4, 3)), ('ruse', (4, 3)), ('lays', (4, 3)), ('vill', (3, 3)), ('whit', (3, 3)), ('gasp', (3, 3)), ('hors', (3, 3)), ('mews', (3, 3)), ('rams', (3, 3)), ('twos', (3, 3)), ('mice', (3, 3)), ('bust', (3, 3)), ('films', (3, 3)), ('hides', (3, 3)), ('cows', (3, 3)), ('emit', (3, 3)), ('ami', (3, 3)), ('girt', (3, 3)), ('sacs', (3, 3)), ('silks', (3, 3)), ('hilt', (3, 3)), ('discs', (3, 3)), ('wick', (3, 3)), ('dire', (3, 3)), ('pots', (3, 3)), ('dint', (3, 3)), ('spas', (3, 3)), ('rugs', (3, 3)), ('airy', (3, 3)), ('epi', (3, 3)), ('tire', (3, 3)), ('joins', (3, 3)), ('fain', (3, 3)), ('lice', (3, 3)), ('dues', (3, 3)), ('mars', (3, 3)), ('efim', (3, 3)), ('lion', (3, 3)), ('spin', (3, 3)), ('twigs', (3, 3)), ('nile', (3, 3)), ('situ', (3, 3)), ('mimi', (3, 3)), ('bipp', (3, 3)), ('tops', (3, 3)), ('rift', (3, 3)), ('gods', (3, 3)), ('pier', (3, 3)), ('bail', (3, 3)), ('twas', (3, 3)), ('cabs', (3, 3)), ('foi', (3, 3)), ('zest', (3, 3)), ('puts', (3, 3)), ('oaks', (3, 3)), ('glio', (3, 3)), ('dites', (3, 3)), ('didst', (3, 3)), ('peas', (3, 3)), ('sash', (3, 3)), ('nasi', (3, 3)), ('naso', (3, 3)), ('nuns', (2, 3)), ('bait', (2, 3)), ('ossis', (2, 3)), ('wily', (2, 3)), ('sur', (2, 3)), ('mints', (2, 3)), ('tags', (2, 3)), ('puris', (2, 3)), ('lily', (2, 3)), ('toys', (2, 3)), ('xxix', (2, 3)), ('sous', (2, 3)), ('piers', (2, 3)), ('slid', (2, 3)), ('jails', (2, 3)), ('viens', (2, 3)), ('alps', (2, 3)), ('amie', (2, 3)), ('bien', (2, 3)), ('fads', (2, 3)), ('docs', (2, 3)), ('bons', (2, 3)), ('gust', (2, 3)), ('gash', (2, 3)), ('wags', (2, 3)), ('wires', (2, 3)), ('mats', (2, 3)), ('aisle', (2, 3)), ('crib', (2, 3)), ('rats', (2, 3)), ('asch', (2, 3)), ('stirs', (2, 3)), ('lick', (2, 3)), ('sal', (2, 3)), ('muir', (2, 3)), ('gibe', (2, 3)), ('sap', (2, 3)), ('etes', (2, 3)), ('lifts', (2, 3)), ('bets', (2, 3)), ('pail', (2, 3)), ('coils', (2, 3)), ('hofs', (2, 3)), ('mien', (2, 3)), ('mots', (2, 3)), ('prim', (2, 3)), ('dice', (2, 3)), ('silas', (2, 3)), ('sila', (2, 3)), ('titus', (2, 3)), ('nidus', (2, 3)), ('idly', (2, 3)), ('jobs', (2, 3)), ('sov', (2, 3)), ('idol', (2, 3)), ('spit', (2, 3)), ('nays', (2, 3)), ('wilt', (2, 3)), ('vows', (2, 3)), ('rimes', (2, 3)), ('swish', (2, 3)), ('wisps', (2, 3)), ('drip', (2, 3)), ('rife', (2, 3)), ('vices', (2, 3)), ('kills', (2, 3)), ('sizes', (2, 3)), ('hogs', (2, 3)), ('risus', (2, 3)), ('weir', (2, 3)), ('ashy', (2, 3)), ('ores', (2, 3)), ('wilds', (2, 3)), ('whims', (2, 3)), ('cats', (2, 3)), ('toi', (2, 3)), ('clips', (2, 3)), ('rust', (2, 3)), ('yeas', (2, 3)), ('esch', (2, 3)), ('mash', (2, 3)), ('vista', (2, 3)), ('rains', (2, 3)), ('misha', (2, 3)), ('sails', (2, 3)), ('hives', (2, 3)), ('nods', (2, 3)), ('mast', (2, 3)), ('ali', (1, 3)), ('sca', (1, 3)), ('muss', (1, 3)), ('bike', (1, 3)), ('sens', (1, 3)), ('vase', (1, 3)), ('voit', (1, 3)), ('pious', (1, 3)), ('gipsy', (1, 3)), ('limes', (1, 3)), ('whist', (1, 3)), ('naiv', (1, 3)), ('tires', (1, 3)), ('elise', (1, 3)), ('pubis', (1, 3)), ('cosy', (1, 3)), ('sup', (1, 3)), ('firms', (1, 3)), ('sinks', (1, 3)), ('chins', (1, 3)), ('epic', (1, 3)), ('kirk', (1, 3)), ('hasp', (1, 3)), ('maim', (1, 3)), ('saws', (1, 3)), ('paws', (1, 3)), ('gosp', (1, 3)), ('sird', (1, 3)), ('hans', (1, 3)), ('isles', (1, 3)), ('icel', (1, 3)), ('lush', (1, 3)), ('muse', (1, 3)), ('kvas', (1, 3)), ('nina', (1, 3)), ('epics', (1, 3)), ('quos', (1, 3)), ('tilt', (1, 3)), ('wiles', (1, 3)), ('vasa', (1, 3)), ('dive', (1, 3)), ('gase', (1, 3)), ('vlas', (1, 3)), ('cain', (1, 3)), ('tile', (1, 3)), ('alias', (1, 3)), ('ilii', (1, 3)), ('pans', (1, 3)), ('rire', (1, 3)), ('nets', (1, 3)), ('licks', (1, 3)), ('cusp', (1, 3)), ('wast', (1, 3)), ('fogs', (1, 3)), ('edin', (1, 3)), ('olin', (1, 3)), ('woes', (1, 3)), ('irons', (1, 3)), ('cups', (1, 3)), ('lair', (1, 3)), ('gush', (1, 3)), ('oasis', (1, 3)), ('quits', (1, 3)), ('riser', (1, 3)), ('limps', (1, 3)), ('bask', (1, 3)), ('fiat', (1, 3)), ('grist', (1, 3)), ('thins', (1, 3)), ('dirk', (1, 3)), ('mesh', (1, 3)), ('daisy', (1, 3)), ('sew', (1, 3)), ('preis', (1, 3)), ('jugs', (1, 3)), ('uric', (1, 3)), ('missy', (1, 3)), ('siam', (1, 3)), ('zeus', (1, 3)), ('grms', (1, 3)), ('sta', (1, 3)), ('iota', (1, 3)), ('binds', (1, 3)), ('liffs', (1, 3)), ('minus', (1, 3)), ('risky', (1, 3)), ('sein', (1, 3)), ('waise', (1, 3)), ('swims', (1, 3)), ('dial', (1, 3)), ('mein', (1, 3)), ('kite', (1, 3)), ('sows', (1, 3)), ('mains', (1, 3)), ('liza', (1, 3)), ('dick', (1, 3)), ('robs', (1, 3)), ('skits', (1, 3)), ('mike', (1, 3)), ('owes', (1, 3)), ('hays', (1, 3)), ('chips', (1, 3)), ('gras', (1, 3)), ('fixes', (1, 3)), ('disco', (1, 3)), ('reid', (1, 3)), ('baits', (1, 3)), ('rives', (1, 3)), ('wiry', (1, 3)), ('vine', (1, 3)), ('sez', (1, 3)), ('hoes', (1, 3)), ('rifts', (1, 3)), ('ibid', (1, 3)), ('wirt', (1, 3)), ('pets', (1, 3)), ('waps', (1, 3)), ('diese', (1, 3)), ('tiers', (1, 3)), ('cite', (1, 3)), ('oars', (1, 3)), ('denis', (1, 3)), ('twig', (1, 3)), ('quid', (1, 3)), ('tiens', (1, 3)), ('frise', (1, 3)), ('dews', (1, 3)), ('isaac', (1, 3)), ('fess', (1, 3)), ('rubs', (1, 3)), ('kicks', (1, 3)), ('wiki', (1, 3)), ('hubs', (1, 3)), ('soie', (1, 3)), ('soils', (1, 3)), ('sitz', (1, 3)), ('chi', (1, 3)), ('eric', (1, 3)), ('dots', (1, 3)), ('gros', (1, 3)), ('ness', (1, 3)), ('vilas', (1, 3)), ('fiew', (1, 3)), ('brig', (1, 3)), ('dost', (1, 3)), ('jose', (1, 3)), ('rosa', (1, 3)), ('slits', (1, 3)), ('ski', (1, 3)), ('pill', (1, 3)), ('skies', (1, 3)), ('grit', (1, 3)), ('sty', (1, 3)), ('fats', (1, 3)), ('tick', (1, 3)), ('cogs', (1, 3)), ('mint', (1, 3)), ('disks', (1, 3)), ('amos', (1, 3)), ('pals', (1, 3)), ('finns', (1, 3)), ('titi', (1, 3)), ('oui', (1, 3)), ('rien', (1, 3)), ('ellis', (1, 3)), ('loins', (1, 3)), ('sots', (1, 3)), ('brims', (1, 3)), ('zides', (1, 3)), ('weib', (1, 3)), ('grid', (1, 3)), ('awls', (1, 3)), ('sol', (1, 3)), ('loin', (1, 3)), ('sot', (1, 3)), ('waits', (1, 3)), ('loi', (1, 3)), ('lions', (1, 3)), ('tints', (1, 3)), ('gibb', (1, 3)), ('linz', (1, 3)), ('lows', (1, 3)), ('viva', (1, 3)), ('mists', (1, 3)), ('gird', (1, 3)), ('gaps', (1, 3)), ('pest', (1, 3)), ('bide', (1, 3)), ('trips', (1, 3)), ('phil', (1, 3)), ('chas', (1, 3)), ('shit', (1, 3)), ('pails', (1, 3)), ('nick', (1, 3)), ('tides', (1, 3)), ('foil', (1, 3)), ('chris', (1, 3)), ('foie', (1, 3)), ('iodi', (1, 3)), ('odds', (1, 3)), ('acids', (1, 3)), ('vines', (1, 3))]
# [('ax', (11, 0)), ('a', (21155, 1)), ('as', (8064, 1)), ('at', (6791, 1)), ('an', (3423, 1)), ('am', (746, 1)), ('ah', (222, 1)), ('x', (136, 1)), ('tax', (84, 1)), ('ix', (28, 1)), ('wax', (22, 1)), ('ex', (18, 1)), ('au', (18, 1)), ('xx', (14, 1)), ('ai', (3, 1)), ('ak', (3, 1)), ('ay', (3, 1)), ('aux', (2, 1)), ('max', (2, 1)), ('lax', (2, 1)), ('aix', (2, 1)), ('ox', (2, 1)), ('ab', (2, 1)), ('fax', (1, 1)), ('axe', (1, 1)), ('ad', (1, 1)), ('al', (1, 1)), ('ac', (1, 1)), ('ar', (1, 1)), ('and', (38313, 2)), ('was', (11410, 2)), ('had', (7383, 2)), ('all', (4144, 2)), ('are', (3630, 2)), ('may', (2551, 2)), ('man', (1652, 2)), ('has', (1603, 2)), ('any', (1204, 2)), ('can', (1095, 2)), ('war', (881, 2)), ('way', (859, 2)), ('day', (819, 2)), ('say', (755, 2)), ('saw', (599, 2)), ('law', (432, 2)), ('far', (408, 2)), ('sat', (403, 2)), ('ran', (322, 2)), ('act', (321, 2)), ('lay', (295, 2)), ('arm', (262, 2)), ('ask', (251, 2)), ('air', (228, 2)), ('six', (176, 2)), ('bad', (155, 2)), ('age', (137, 2)), ('pay', (128, 2)), ('aid', (113, 2)), ('aim', (108, 2)), ('ago', (108, 2)), ('hat', (105, 2)), ('cap', (97, 2)), ('sad', (91, 2)), ('fat', (89, 2)), ('box', (75, 2)), ('ha', (75, 2)), ('sac', (73, 2)), ('lad', (70, 2)), ('raw', (58, 2)), ('la', (55, 2)), ('eat', (54, 2)), ('art', (47, 2)), ('gap', (47, 2)), ('ear', (46, 2)), ('hay', (42, 2)), ('aet', (40, 2)), ('ray', (40, 2)), ('map', (38, 2)), ('axis', (37, 2)), ('mad', (36, 2)), ('jaw', (36, 2)), ('apt', (35, 2)), ('gay', (35, 2)), ('cab', (33, 2)), ('ma', (31, 2)), ('dam', (30, 2)), ('oak', (30, 2)), ('xi', (28, 2)), ('fix', (27, 2)), ('add', (27, 2)), ('bar', (25, 2)), ('xv', (24, 2)), ('bay', (23, 2)), ('fox', (23, 2)), ('pas', (23, 2)), ('ate', (21, 2)), ('bag', (21, 2)), ('san', (20, 2)), ('pad', (20, 2)), ('van', (19, 2)), ('jay', (18, 2)), ('xix', (17, 2)), ('gas', (16, 2)), ('pa', (15, 2)), ('rat', (13, 2)), ('xxi', (13, 2)), ('awe', (11, 2)), ('sex', (11, 2)), ('lap', (11, 2)), ('cat', (10, 2)), ('tap', (10, 2)), ('fan', (8, 2)), ('txt', (8, 2)), ('nap', (8, 2)), ('va', (7, 2)), ('car', (7, 2)), ('tar', (7, 2)), ('dad', (7, 2)), ('waxy', (7, 2)), ('ram', (7, 2)), ('xxv', (7, 2)), ('saxe', (7, 2)), ('ta', (6, 2)), ('flax', (6, 2)), ('zat', (6, 2)), ('rag', (6, 2)), ('sam', (6, 2)), ('ban', (6, 2)), ('ham', (6, 2)), ('axes', (5, 2)), ('ash', (5, 2)), ('jar', (5, 2)), ('arc', (5, 2)), ('par', (5, 2)), ('wag', (5, 2)), ('ape', (4, 2)), ('sa', (4, 2)), ('apex', (4, 2)), ('da', (4, 2)), ('pal', (4, 2)), ('pan', (4, 2)), ('mix', (4, 2)), ('vex', (3, 2)), ('jam', (3, 2)), ('mat', (3, 2)), ('mal', (3, 2)), ('ant', (3, 2)), ('pat', (3, 2)), ('ame', (3, 2)), ('aye', (3, 2)), ('eau', (3, 2)), ('ami', (3, 2)), ('na', (3, 2)), ('nay', (3, 2)), ('abe', (2, 2)), ('tag', (2, 2)), ('dan', (2, 2)), ('bat', (2, 2)), ('ass', (2, 2)), ('bah', (2, 2)), ('ann', (2, 2)), ('dat', (2, 2)), ('bal', (2, 2)), ('cox', (2, 2)), ('ana', (2, 2)), ('sal', (2, 2)), ('xxx', (2, 2)), ('gag', (2, 2)), ('sap', (2, 2)), ('paw', (2, 2)), ('ba', (2, 2)), ('wad', (2, 2)), ('nag', (2, 2)), ('ali', (1, 2)), ('ga', (1, 2)), ('ale', (1, 2)), ('fas', (1, 2)), ('fad', (1, 2)), ('dix', (1, 2)), ('asp', (1, 2)), ('aug', (1, 2)), ('mac', (1, 2)), ('cal', (1, 2)), ('tat', (1, 2)), ('aft', (1, 2)), ('lab', (1, 2)), ('coax', (1, 2)), ('ra', (1, 2)), ('att', (1, 2)), ('las', (1, 2)), ('vat', (1, 2)), ('ano', (1, 2)), ('ras', (1, 2)), ('ca', (1, 2)), ('abc', (1, 2)), ('jan', (1, 2)), ('ark', (1, 2)), ('adv', (1, 2)), ('wa', (1, 2)), ('ja', (1, 2)), ('pox', (1, 2)), ('marx', (1, 2)), ('gal', (1, 2)), ('rah', (1, 2)), ('adj', (1, 2)), ('ain', (1, 2)), ('hoax', (1, 2)), ('ave', (1, 2)), ('aah', (1, 2)), ('taxi', (1, 2)), ('nan', (1, 2)), ('das', (1, 2)), ('amp', (1, 2)), ('ado', (1, 2)), ('amy', (1, 2)), ('ace', (1, 2)), ('awl', (1, 2)), ('wan', (1, 2)), ('val', (1, 2)), ('vas', (1, 2)), ('wat', (1, 2)), ('ka', (1, 2)), ('ext', (1, 2)), ('oar', (1, 2)), ('that', (12512, 3)), ('have', (3493, 3)), ('said', (3464, 3)), ('what', (3011, 3)), ('than', (1206, 3)), ('face', (1125, 3)), ('same', (1052, 3)), ('made', (1007, 3)), ('came', (979, 3)), ('away', (838, 3)), ('hand', (834, 3)), ('also', (778, 3)), ('army', (764, 3)), ('back', (746, 3)), ('head', (725, 3)), ('mary', (705, 3)), ('part', (704, 3)), ('take', (616, 3)), ('many', (609, 3)), ('last', (565, 3)), ('make', (504, 3)), ('days', (453, 3)), ('dear', (449, 3)), ('gave', (442, 3)), ('case', (438, 3)), ('each', (411, 3)), ('want', (323, 3)), ('half', (318, 3)), ('year', (309, 3)), ('pain', (303, 3)), ('anna', (294, 3)), ('near', (293, 3)), ('fact', (291, 3)), ('talk', (287, 3)), ('land', (279, 3)), ('next', (277, 3)), ('name', (262, 3)), ('road', (260, 3)), ('arms', (249, 3)), ('hair', (234, 3)), ('laws', (233, 3)), ('past', (223, 3)), ('read', (219, 3)), ('able', (201, 3)), ('call', (197, 3)), ('wall', (190, 3)), ('laid', (186, 3)), ('hear', (183, 3)), ('dark', (181, 3)), ('hard', (180, 3)), ('lady', (177, 3)), ('coat', (172, 3)), ('seat', (170, 3)), ('fear', (166, 3)), ('late', (165, 3)), ('pale', (165, 3)), ('dead', (164, 3)), ('area', (163, 3)), ('plan', (159, 3)), ('pass', (154, 3)), ('glad', (150, 3)), ('says', (146, 3)), ('camp', (138, 3)), ('lead', (137, 3)), ('east', (130, 3)), ('wait', (127, 3)), ('fall', (124, 3)), ('ball', (124, 3)), ('easy', (122, 3)), ('save', (110, 3)), ('main', (109, 3)), ('bank', (109, 3)), ('tea', (107, 3)), ('care', (106, 3)), ('paid', (106, 3)), ('real', (102, 3)), ('mass', (101, 3)), ('path', (100, 3)), ('mean', (98, 3)), ('fate', (98, 3)), ('sake', (97, 3)), ('play', (95, 3)), ('calm', (91, 3)), ('scar', (89, 3)), ('acts', (89, 3)), ('gray', (89, 3)), ('rays', (87, 3)), ('maid', (87, 3)), ('bear', (86, 3)), ('weak', (85, 3)), ('amid', (84, 3)), ('hall', (83, 3)), ('rare', (83, 3)), ('heat', (83, 3)), ('sea', (82, 3)), ('bald', (82, 3)), ('yard', (79, 3)), ('pray', (79, 3)), ('anti', (76, 3)), ('warm', (75, 3)), ('walk', (75, 3)), ('tall', (74, 3)), ('bare', (74, 3)), ('stay', (74, 3)), ('clay', (72, 3)), ('deal', (69, 3)), ('rate', (67, 3)), ('wars', (67, 3)), ('gate', (65, 3)), ('taxes', (63, 3)), ('acid', (61, 3)), ('nail', (60, 3)), ('page', (59, 3)), ('ears', (58, 3)), ('tsar', (56, 3)), ('vary', (56, 3)), ('cart', (56, 3)), ('fair', (55, 3)), ('game', (55, 3)), ('draw', (55, 3)), ('band', (54, 3)), ('cast', (54, 3)), ('base', (53, 3)), ('aunt', (52, 3)), ('ways', (52, 3)), ('farm', (50, 3)), ('papa', (50, 3)), ('hart', (49, 3)), ('task', (49, 3)), ('date', (48, 3)), ('vast', (48, 3)), ('aide', (48, 3)), ('safe', (47, 3)), ('beat', (46, 3)), ('baby', (45, 3)), ('rank', (45, 3)), ('ease', (44, 3)), ('gain', (44, 3)), ('dare', (43, 3)), ('race', (42, 3)), ('trap', (42, 3)), ('harm', (42, 3)), ('tail', (40, 3)), ('pace', (40, 3)), ('male', (40, 3)), ('pair', (40, 3)), ('fail', (40, 3)), ('heal', (40, 3)), ('coal', (39, 3)), ('rain', (39, 3)), ('fast', (39, 3)), ('navy', (38, 3)), ('mark', (38, 3)), ('lads', (38, 3)), ('lamp', (37, 3)), ('seas', (37, 3)), ('taft', (36, 3)), ('hare', (36, 3)), ('sank', (35, 3)), ('sang', (34, 3)), ('flat', (34, 3)), ('salt', (34, 3)), ('xvi', (34, 3)), ('palm', (34, 3)), ('lack', (33, 3)), ('varix', (33, 3)), ('flag', (31, 3)), ('aged', (31, 3)), ('pack', (31, 3)), ('card', (30, 3)), ('wear', (30, 3)), ('vain', (30, 3)), ('rear', (30, 3)), ('rang', (29, 3)), ('damp', (29, 3)), ('goal', (28, 3)), ('xii', (28, 3)), ('xiv', (28, 3)), ('gaze', (28, 3)), ('flap', (27, 3)), ('wake', (27, 3)), ('lake', (27, 3)), ('lane', (27, 3)), ('wave', (26, 3)), ('oath', (26, 3)), ('text', (26, 3)), ('sale', (25, 3)), ('zeal', (25, 3)), ('roar', (25, 3)), ('tear', (24, 3)), ('star', (24, 3)), ('aims', (24, 3)), ('dawn', (22, 3)), ('mack', (22, 3)), ('bath', (22, 3)), ('rage', (22, 3)), ('tale', (21, 3)), ('asks', (21, 3)), ('utah', (21, 3)), ('hate', (20, 3)), ('halt', (20, 3)), ('wash', (20, 3)), ('aren', (19, 3)), ('abbe', (19, 3)), ('caps', (19, 3)), ('jaws', (18, 3)), ('bass', (18, 3)), ('exit', (18, 3)), ('calf', (17, 3)), ('data', (17, 3)), ('barn', (17, 3)), ('arts', (17, 3)), ('hang', (17, 3)), ('wage', (17, 3)), ('heap', (17, 3)), ('sand', (16, 3)), ('ages', (16, 3)), ('paul', (16, 3)), ('shan', (15, 3)), ('meal', (15, 3)), ('park', (15, 3)), ('oats', (15, 3)), ('seal', (15, 3)), ('fame', (15, 3)), ('meat', (15, 3)), ('akin', (14, 3)), ('tact', (14, 3)), ('wasn', (14, 3)), ('lawn', (14, 3)), ('bags', (14, 3)), ('clad', (14, 3)), ('pea', (14, 3)), ('arch', (13, 3)), ('ward', (13, 3)), ('earn', (13, 3)), ('lean', (12, 3)), ('bast', (12, 3)), ('karp', (12, 3)), ('hadn', (12, 3)), ('mask', (12, 3)), ('lace', (11, 3)), ('sway', (11, 3)), ('cash', (11, 3)), ('oval', (11, 3)), ('warn', (11, 3)), ('scab', (11, 3)), ('rail', (11, 3)), ('boat', (11, 3)), ('dram', (11, 3)), ('hats', (10, 3)), ('gang', (10, 3)), ('xxii', (10, 3)), ('diam', (10, 3)), ('era', (10, 3)), ('rapp', (10, 3)), ('ally', (10, 3)), ('moan', (10, 3)), ('karl', (10, 3)), ('loan', (10, 3)), ('deaf', (10, 3)), ('anne', (10, 3)), ('acre', (10, 3)), ('drag', (10, 3)), ('adds', (9, 3)), ('hasn', (9, 3)), ('raft', (9, 3)), ('jail', (9, 3)), ('mail', (9, 3)), ('dash', (9, 3)), ('chap', (9, 3)), ('ivan', (9, 3)), ('bark', (9, 3)), ('mane', (9, 3)), ('anus', (9, 3)), ('wail', (9, 3)), ('snap', (9, 3)), ('rags', (9, 3)), ('chat', (9, 3)), ('jack', (8, 3)), ('team', (8, 3)), ('hast', (8, 3)), ('xxiv', (8, 3)), ('malo', (8, 3)), ('eats', (8, 3)), ('tray', (8, 3)), ('harp', (8, 3)), ('gait', (8, 3)), ('bars', (8, 3)), ('sail', (8, 3)), ('lash', (8, 3)), ('load', (8, 3)), ('neat', (8, 3)), ('sack', (8, 3)), ('stab', (8, 3)), ('hath', (7, 3)), ('cars', (7, 3)), ('maps', (7, 3)), ('fare', (7, 3)), ('bang', (7, 3)), ('gape', (7, 3)), ('slay', (7, 3)), ('soap', (7, 3)), ('rash', (7, 3)), ('pads', (7, 3)), ('klux', (7, 3)), ('raid', (7, 3)), ('leaf', (7, 3)), ('liar', (6, 3)), ('vaso', (6, 3)), ('rack', (6, 3)), ('diaz', (6, 3)), ('arid', (6, 3)), ('klan', (6, 3)), ('saxon', (6, 3)), ('flex', (6, 3)), ('lain', (6, 3)), ('czar', (6, 3)), ('dans', (6, 3)), ('gale', (6, 3)), ('gram', (6, 3)), ('anew', (6, 3)), ('fade', (6, 3)), ('cake', (6, 3)), ('kate', (5, 3)), ('wavy', (5, 3)), ('alms', (5, 3)), ('arab', (5, 3)), ('lame', (5, 3)), ('hawk', (5, 3)), ('mais', (5, 3)), ('cage', (5, 3)), ('mate', (5, 3)), ('taxed', (5, 3)), ('aime', (5, 3)), ('bade', (5, 3)), ('acne', (5, 3)), ('fait', (5, 3)), ('lazy', (5, 3)), ('ants', (5, 3)), ('nape', (5, 3)), ('cane', (5, 3)), ('hoar', (5, 3)), ('haul', (5, 3)), ('claw', (5, 3)), ('mare', (5, 3)), ('pays', (5, 3)), ('oxen', (5, 3)), ('hail', (5, 3)), ('wart', (5, 3)), ('adam', (5, 3)), ('sans', (4, 3)), ('anal', (4, 3)), ('asia', (4, 3)), ('haze', (4, 3)), ('flaw', (4, 3)), ('slab', (4, 3)), ('apes', (4, 3)), ('lata', (4, 3)), ('rake', (4, 3)), ('foam', (4, 3)), ('yale', (4, 3)), ('isa', (4, 3)), ('gear', (4, 3)), ('myxo', (4, 3)), ('faut', (4, 3)), ('gall', (4, 3)), ('yawn', (4, 3)), ('alas', (4, 3)), ('sage', (4, 3)), ('hack', (4, 3)), ('feat', (4, 3)), ('gage', (4, 3)), ('bean', (4, 3)), ('earl', (4, 3)), ('dams', (4, 3)), ('sait', (4, 3)), ('loaf', (4, 3)), ('damn', (4, 3)), ('xxvi', (4, 3)), ('shag', (4, 3)), ('awed', (4, 3)), ('maxim', (4, 3)), ('leap', (4, 3)), ('ache', (4, 3)), ('wrap', (4, 3)), ('span', (4, 3)), ('atom', (4, 3)), ('anel', (4, 3)), ('lays', (4, 3)), ('awry', (4, 3)), ('peak', (4, 3)), ('oaks', (3, 3)), ('via', (3, 3)), ('relax', (3, 3)), ('ware', (3, 3)), ('rapt', (3, 3)), ('goat', (3, 3)), ('tack', (3, 3)), ('gasp', (3, 3)), ('juan', (3, 3)), ('gaol', (3, 3)), ('carl', (3, 3)), ('pang', (3, 3)), ('avec', (3, 3)), ('cady', (3, 3)), ('guam', (3, 3)), ('sacs', (3, 3)), ('airy', (3, 3)), ('spas', (3, 3)), ('slam', (3, 3)), ('cave', (3, 3)), ('cava', (3, 3)), ('fain', (3, 3)), ('axiom', (3, 3)), ('beam', (3, 3)), ('bead', (3, 3)), ('halo', (3, 3)), ('balk', (3, 3)), ('weal', (3, 3)), ('rams', (3, 3)), ('afar', (3, 3)), ('ante', (3, 3)), ('airs', (3, 3)), ('mars', (3, 3)), ('davy', (3, 3)), ('dual', (3, 3)), ('auto', (3, 3)), ('kalb', (3, 3)), ('mayo', (3, 3)), ('cape', (3, 3)), ('bail', (3, 3)), ('twas', (3, 3)), ('cabs', (3, 3)), ('watt', (3, 3)), ('dame', (3, 3)), ('wary', (3, 3)), ('pear', (3, 3)), ('peas', (3, 3)), ('sash', (3, 3)), ('nasi', (3, 3)), ('naso', (3, 3)), ('hata', (2, 3)), ('swab', (2, 3)), ('swam', (2, 3)), ('swan', (2, 3)), ('cahd', (2, 3)), ('xxix', (2, 3)), ('tags', (2, 3)), ('sawn', (2, 3)), ('pawn', (2, 3)), ('crab', (2, 3)), ('waxen', (2, 3)), ('babe', (2, 3)), ('flay', (2, 3)), ('warp', (2, 3)), ('alba', (2, 3)), ('yarn', (2, 3)), ('rape', (2, 3)), ('lamb', (2, 3)), ('xxxi', (2, 3)), ('okay', (2, 3)), ('fray', (2, 3)), ('amie', (2, 3)), ('vara', (2, 3)), ('pane', (2, 3)), ('fads', (2, 3)), ('acme', (2, 3)), ('acte', (2, 3)), ('tra', (2, 3)), ('kann', (2, 3)), ('hark', (2, 3)), ('gash', (2, 3)), ('yeah', (2, 3)), ('yeas', (2, 3)), ('aids', (2, 3)), ('mats', (2, 3)), ('rats', (2, 3)), ('cats', (2, 3)), ('alps', (2, 3)), ('thaw', (2, 3)), ('parr', (2, 3)), ('dank', (2, 3)), ('asch', (2, 3)), ('slap', (2, 3)), ('quay', (2, 3)), ('bake', (2, 3)), ('pail', (2, 3)), ('oka', (2, 3)), ('bait', (2, 3)), ('tape', (2, 3)), ('beau', (2, 3)), ('beak', (2, 3)), ('gaul', (2, 3)), ('hayn', (2, 3)), ('wags', (2, 3)), ('coxa', (2, 3)), ('nays', (2, 3)), ('tank', (2, 3)), ('bias', (2, 3)), ('sham', (2, 3)), ('dart', (2, 3)), ('soak', (2, 3)), ('reap', (2, 3)), ('knox', (2, 3)), ('shaw', (2, 3)), ('lank', (2, 3)), ('pall', (2, 3)), ('mead', (2, 3)), ('pago', (2, 3)), ('peal', (2, 3)), ('roan', (2, 3)), ('ashy', (2, 3)), ('ida', (2, 3)), ('jane', (2, 3)), ('spat', (2, 3)), ('roam', (2, 3)), ('tafa', (2, 3)), ('mash', (2, 3)), ('dale', (2, 3)), ('mast', (2, 3)), ('oral', (2, 3)), ('axles', (1, 3)), ('sca', (1, 3)), ('veux', (1, 3)), ('swag', (1, 3)), ('grab', (1, 3)), ('kvas', (1, 3)), ('ague', (1, 3)), ('vasa', (1, 3)), ('yeux', (1, 3)), ('eva', (1, 3)), ('naiv', (1, 3)), ('gase', (1, 3)), ('rath', (1, 3)), ('lea', (1, 3)), ('sexe', (1, 3)), ('dahe', (1, 3)), ('vale', (1, 3)), ('pate', (1, 3)), ('hasp', (1, 3)), ('alte', (1, 3)), ('boa', (1, 3)), ('paws', (1, 3)), ('ajar', (1, 3)), ('raum', (1, 3)), ('sane', (1, 3)), ('khan', (1, 3)), ('swap', (1, 3)), ('adve', (1, 3)), ('azur', (1, 3)), ('waxed', (1, 3)), ('malt', (1, 3)), ('mall', (1, 3)), ('taut', (1, 3)), ('mala', (1, 3)), ('scan', (1, 3)), ('andy', (1, 3)), ('xxxv', (1, 3)), ('faro', (1, 3)), ('arcy', (1, 3)), ('dato', (1, 3)), ('vlas', (1, 3)), ('kari', (1, 3)), ('pans', (1, 3)), ('wead', (1, 3)), ('waps', (1, 3)), ('lava', (1, 3)), ('agar', (1, 3)), ('maim', (1, 3)), ('sate', (1, 3)), ('safi', (1, 3)), ('bane', (1, 3)), ('hale', (1, 3)), ('guai', (1, 3)), ('beaux', (1, 3)), ('alma', (1, 3)), ('noxa', (1, 3)), ('matt', (1, 3)), ('lair', (1, 3)), ('carp', (1, 3)), ('pua', (1, 3)), ('bask', (1, 3)), ('fiat', (1, 3)), ('cain', (1, 3)), ('saul', (1, 3)), ('wast', (1, 3)), ('agra', (1, 3)), ('tate', (1, 3)), ('avez', (1, 3)), ('siam', (1, 3)), ('amen', (1, 3)), ('para', (1, 3)), ('vase', (1, 3)), ('pari', (1, 3)), ('slav', (1, 3)), ('sta', (1, 3)), ('dane', (1, 3)), ('roux', (1, 3)), ('deux', (1, 3)), ('takh', (1, 3)), ('soar', (1, 3)), ('dial', (1, 3)), ('plat', (1, 3)), ('lafa', (1, 3)), ('ayez', (1, 3)), ('hays', (1, 3)), ('jazz', (1, 3)), ('gras', (1, 3)), ('graf', (1, 3)), ('bart', (1, 3)), ('avow', (1, 3)), ('exam', (1, 3)), ('alum', (1, 3)), ('aloe', (1, 3)), ('opal', (1, 3)), ('anon', (1, 3)), ('brag', (1, 3)), ('mann', (1, 3)), ('oars', (1, 3)), ('loam', (1, 3)), ('walt', (1, 3)), ('marm', (1, 3)), ('dave', (1, 3)), ('brat', (1, 3)), ('alae', (1, 3)), ('saws', (1, 3)), ('arat', (1, 3)), ('foxy', (1, 3)), ('shak', (1, 3)), ('shah', (1, 3)), ('wand', (1, 3)), ('foal', (1, 3)), ('mawr', (1, 3)), ('hans', (1, 3)), ('fats', (1, 3)), ('veal', (1, 3)), ('jean', (1, 3)), ('noah', (1, 3)), ('yea', (1, 3)), ('agog', (1, 3)), ('ably', (1, 3)), ('jake', (1, 3)), ('clan', (1, 3)), ('amoy', (1, 3)), ('amos', (1, 3)), ('deah', (1, 3)), ('dean', (1, 3)), ('pals', (1, 3)), ('alan', (1, 3)), ('leak', (1, 3)), ('saar', (1, 3)), ('lark', (1, 3)), ('cafe', (1, 3)), ('aden', (1, 3)), ('awls', (1, 3)), ('gare', (1, 3)), ('garb', (1, 3)), ('wade', (1, 3)), ('abel', (1, 3)), ('tame', (1, 3)), ('heah', (1, 3)), ('azor', (1, 3)), ('azov', (1, 3)), ('gaps', (1, 3)), ('chas', (1, 3)), ('etat', (1, 3)), ('annex', (1, 3)), ('stag', (1, 3)), ('drab', (1, 3)), ('joan', (1, 3))]
# [('test', (53, 0)), ('west', (286, 1)), ('best', (268, 1)), ('rest', (209, 1)), ('est', (29, 1)), ('text', (26, 1)), ('lest', (25, 1)), ('tent', (24, 1)), ('jest', (17, 1)), ('nest', (7, 1)), ('tests', (7, 1)), ('vest', (5, 1)), ('zest', (3, 1)), ('pest', (1, 1)), ('that', (12512, 2)), ('these', (1231, 2)), ('went', (1008, 2)), ('must', (955, 2)), ('most', (908, 2)), ('left', (834, 2)), ('just', (767, 2)), ('felt', (697, 2)), ('yes', (688, 2)), ('last', (565, 2)), ('met', (544, 2)), ('let', (506, 2)), ('tell', (492, 2)), ('yet', (488, 2)), ('get', (468, 2)), ('less', (367, 2)), ('set', (325, 2)), ('sent', (319, 2)), ('next', (277, 2)), ('kept', (251, 2)), ('lost', (224, 2)), ('past', (223, 2)), ('ten', (219, 2)), ('least', (194, 2)), ('feet', (179, 2)), ('meet', (170, 2)), ('seat', (170, 2)), ('st', (147, 2)), ('term', (133, 2)), ('east', (130, 2)), ('post', (117, 2)), ('tea', (107, 2)), ('bent', (98, 2)), ('heat', (83, 2)), ('chest', (81, 2)), ('cent', (77, 2)), ('teeth', (76, 2)), ('trust', (68, 2)), ('cyst', (67, 2)), ('cost', (65, 2)), ('debt', (65, 2)), ('wet', (60, 2)), ('cast', (54, 2)), ('eat', (54, 2)), ('tend', (52, 2)), ('toes', (51, 2)), ('task', (49, 2)), ('vast', (48, 2)), ('list', (48, 2)), ('beat', (46, 2)), ('mist', (41, 2)), ('aet', (40, 2)), ('dust', (39, 2)), ('fast', (39, 2)), ('taft', (36, 2)), ('treat', (31, 2)), ('net', (28, 2)), ('wept', (27, 2)), ('beast', (26, 2)), ('tear', (24, 2)), ('ties', (24, 2)), ('host', (23, 2)), ('tense', (23, 2)), ('taste', (23, 2)), ('trot', (22, 2)), ('gets', (22, 2)), ('lent', (21, 2)), ('et', (19, 2)), ('tenth', (19, 2)), ('rent', (19, 2)), ('les', (18, 2)), ('rests', (17, 2)), ('guest', (17, 2)), ('bet', (17, 2)), ('latest', (17, 2)), ('pet', (17, 2)), ('tested', (16, 2)), ('tens', (16, 2)), ('meat', (15, 2)), ('quest', (15, 2)), ('tact', (14, 2)), ('twist', (14, 2)), ('teno', (13, 2)), ('bast', (12, 2)), ('tout', (12, 2)), ('belt', (12, 2)), ('vent', (12, 2)), ('tut', (11, 2)), ('sets', (11, 2)), ('desk', (11, 2)), ('tint', (11, 2)), ('des', (10, 2)), ('mess', (10, 2)), ('tents', (9, 2)), ('toast', (9, 2)), ('tete', (9, 2)), ('hast', (8, 2)), ('txt', (8, 2)), ('team', (8, 2)), ('feast', (8, 2)), ('fist', (8, 2)), ('sect', (8, 2)), ('neat', (8, 2)), ('crest', (7, 2)), ('texts', (7, 2)), ('beset', (7, 2)), ('lets', (7, 2)), ('tit', (7, 2)), ('etext', (6, 2)), ('melt', (5, 2)), ('kent', (5, 2)), ('lust', (5, 2)), ('tis', (5, 2)), ('jests', (5, 2)), ('toss', (4, 2)), ('tuft', (4, 2)), ('jet', (4, 2)), ('tres', (4, 2)), ('feat', (4, 2)), ('jets', (4, 2)), ('es', (3, 2)), ('testis', (3, 2)), ('bust', (3, 2)), ('theft', (3, 2)), ('tenn', (3, 2)), ('gist', (3, 2)), ('esq', (3, 2)), ('tease', (3, 2)), ('ted', (2, 2)), ('nests', (2, 2)), ('inst', (2, 2)), ('tempt', (2, 2)), ('etes', (2, 2)), ('bets', (2, 2)), ('gust', (2, 2)), ('ses', (2, 2)), ('deft', (2, 2)), ('welt', (2, 2)), ('pes', (2, 2)), ('sept', (2, 2)), ('rust', (2, 2)), ('mast', (2, 2)), ('tt', (1, 2)), ('te', (1, 2)), ('vert', (1, 2)), ('trent', (1, 2)), ('detest', (1, 2)), ('testes', (1, 2)), ('taut', (1, 2)), ('tat', (1, 2)), ('brest', (1, 2)), ('tilt', (1, 2)), ('mesh', (1, 2)), ('nets', (1, 2)), ('pent', (1, 2)), ('eut', (1, 2)), ('tss', (1, 2)), ('cet', (1, 2)), ('ces', (1, 2)), ('blest', (1, 2)), ('etat', (1, 2)), ('ist', (1, 2)), ('wast', (1, 2)), ('wert', (1, 2)), ('ext', (1, 2)), ('pets', (1, 2)), ('twot', (1, 2)), ('fess', (1, 2)), ('terse', (1, 2)), ('ness', (1, 2)), ('dost', (1, 2)), ('vests', (1, 2)), ('esp', (1, 2)), ('pelt', (1, 2)), ('hatest', (1, 2)), ('the', (80030, 3)), ('to', (28766, 3)), ('he', (12401, 3)), ('was', (11410, 3)), ('it', (10681, 3)), ('his', (10034, 3)), ('is', (9774, 3)), ('as', (8064, 3)), ('at', (6791, 3)), ('not', (6626, 3)), ('be', (6155, 3)), ('but', (5653, 3)), ('s', (5636, 3)), ('her', (5284, 3)), ('were', (4289, 3)), ('this', (4063, 3)), ('they', (3938, 3)), ('so', (3017, 3)), ('what', (3011, 3)), ('there', (2972, 3)), ('their', (2955, 3)), ('been', (2599, 3)), ('them', (2241, 3)), ('out', (1987, 3)), ('me', (1920, 3)), ('we', (1906, 3)), ('its', (1635, 3)), ('has', (1603, 3)), ('then', (1558, 3)), ('time', (1529, 3)), ('very', (1340, 3)), ('t', (1318, 3)), ('new', (1211, 3)), ('than', (1206, 3)), ('those', (1201, 3)), ('well', (1198, 3)), ('first', (1177, 3)), ('men', (1145, 3)), ('two', (1138, 3)), ('see', (1101, 3)), ('eyes', (939, 3)), ('great', (792, 3)), ('also', (778, 3)), ('head', (725, 3)), ('part', (704, 3)), ('here', (691, 3)), ('us', (684, 3)), ('take', (616, 3)), ('took', (573, 3)), ('too', (548, 3)), ('told', (490, 3)), ('end', (465, 3)), ('few', (458, 3)), ('dear', (449, 3)), ('seen', (444, 3)), ('case', (438, 3)), ('put', (435, 3)), ('sat', (403, 3)), ('result', (386, 3)), ('does', (358, 3)), ('death', (330, 3)), ('want', (323, 3)), ('act', (321, 3)), ('use', (320, 3)), ('year', (309, 3)), ('near', (293, 3)), ('fact', (291, 3)), ('red', (288, 3)), ('talk', (287, 3)), ('held', (287, 3)), ('got', (280, 3)), ('used', (276, 3)), ('heart', (256, 3)), ('ask', (251, 3)), ('wish', (243, 3)), ('rose', (243, 3)), ('rise', (240, 3)), ('de', (237, 3)), ('times', (236, 3)), ('help', (230, 3)), ('news', (224, 3)), ('bed', (221, 3)), ('read', (219, 3)), ('self', (218, 3)), ('till', (216, 3)), ('deep', (215, 3)), ('thus', (212, 3)), ('true', (205, 3)), ('neck', (203, 3)), ('else', (201, 3)), ('foot', (200, 3)), ('cut', (199, 3)), ('led', (196, 3)), ('re', (189, 3)), ('turn', (188, 3)), ('steps', (188, 3)), ('soft', (185, 3)), ('hear', (183, 3)), ('fell', (177, 3)), ('town', (174, 3)), ('coat', (172, 3)), ('tears', (172, 3)), ('thin', (166, 3)), ('fear', (166, 3)), ('tone', (166, 3)), ('dead', (164, 3)), ('feel', (161, 3)), ('need', (160, 3)), ('fresh', (160, 3)), ('pus', (155, 3)), ('pass', (154, 3)), ('takes', (153, 3)), ('ve', (153, 3)), ('terms', (148, 3)), ('leg', (145, 3)), ('loss', (143, 3)), ('keep', (142, 3)), ('step', (139, 3)), ('dress', (138, 3)), ('lead', (137, 3)), ('e', (136, 3)), ('tm', (128, 3)), ('wait', (127, 3)), ('easy', (122, 3)), ('hot', (119, 3)), ('legs', (118, 3)), ('seem', (116, 3)), ('vein', (116, 3)), ('truth', (115, 3)), ('meant', (113, 3)), ('miss', (112, 3)), ('event', (111, 3)), ('spent', (111, 3)), ('eye', (110, 3)), ('hat', (105, 3)), ('nose', (103, 3)), ('sense', (103, 3)), ('real', (102, 3)), ('mass', (101, 3)), ('send', (99, 3)), ('mean', (98, 3)), ('berg', (98, 3)), ('week', (95, 3)), ('shot', (95, 3)), ('sort', (91, 3)), ('fat', (89, 3)), ('sit', (89, 3)), ('acts', (89, 3)), ('eh', (89, 3)), ('try', (87, 3)), ('type', (87, 3)), ('bear', (86, 3)), ('breast', (86, 3)), ('per', (86, 3)), ('lies', (86, 3)), ('weak', (85, 3)), ('tax', (84, 3)), ('sea', (82, 3)), ('visit', (81, 3)), ('press', (81, 3)), ('mere', (79, 3)), ('ones', (77, 3)), ('forest', (76, 3)), ('spot', (76, 3)), ('risk', (75, 3)), ('vera', (75, 3)), ('tall', (74, 3)), ('lit', (74, 3)), ('burst', (73, 3)), ('guests', (72, 3)), ('clot', (70, 3)), ('deal', (69, 3)), ('wrist', (68, 3)), ('start', (67, 3)), ('cysts', (66, 3)), ('stern', (65, 3)), ('bell', (65, 3)), ('notes', (64, 3)), ('moist', (63, 3)), ('taxes', (63, 3)), ('texas', (63, 3)), ('bit', (63, 3)), ('stout', (62, 3)), ('debts', (62, 3)), ('votes', (60, 3)), ('goes', (60, 3)), ('torn', (60, 3)), ('arrest', (60, 3)), ('mrs', (59, 3)), ('hut', (58, 3)), ('tsar', (56, 3)), ('beg', (56, 3)), ('cart', (56, 3)), ('busy', (55, 3)), ('hero', (55, 3)), ('le', (55, 3)), ('contest', (54, 3)), ('base', (53, 3)), ('fit', (52, 3)), ('isn', (52, 3)), ('aunt', (52, 3)), ('th', (51, 3)), ('trees', (51, 3)), ('lose', (51, 3)), ('midst', (50, 3)), ('hart', (49, 3)), ('rates', (48, 3)), ('tends', (48, 3)), ('kiss', (47, 3)), ('art', (47, 3)), ('thy', (47, 3)), ('ear', (46, 3)), ('tube', (45, 3)), ('dense', (44, 3)), ('ease', (44, 3)), ('exist', (44, 3)), ('trap', (42, 3)), ('top', (42, 3)), ('tree', (42, 3)), ('thou', (42, 3)), ('tail', (40, 3)), ('shut', (40, 3)), ('tied', (40, 3)), ('coast', (40, 3)), ('heal', (40, 3)), ('doesn', (40, 3)), ('swept', (38, 3)), ('seek', (38, 3)), ('cease', (38, 3)), ('wheat', (38, 3)), ('deed', (37, 3)), ('onset', (37, 3)), ('seas', (37, 3)), ('frost', (37, 3)), ('worst', (36, 3)), ('toe', (36, 3)), ('apt', (35, 3)), ('slept', (35, 3)), ('sweet', (35, 3)), ('lot', (35, 3)), ('rush', (35, 3)), ('flat', (34, 3)), ('hurt', (34, 3)), ('salt', (34, 3)), ('gates', (34, 3)), ('peri', (33, 3)), ('types', (33, 3)), ('tide', (33, 3)), ('resist', (33, 3)), ('keen', (32, 3)), ('priest', (32, 3)), ('lift', (32, 3)), ('tones', (32, 3)), ('hunt', (31, 3)), ('eve', (30, 3)), ('haste', (30, 3)), ('lise', (30, 3)), ('steel', (30, 3)), ('sell', (30, 3)), ('steam', (30, 3)), ('wear', (30, 3)), ('hers', (30, 3)), ('rosy', (30, 3)), ('rear', (30, 3)), ('tight', (30, 3)), ('depth', (30, 3)), ('eldest', (29, 3)), ('toll', (29, 3)), ('fee', (29, 3)), ('sheet', (29, 3)), ('rested', (28, 3)), ('teach', (27, 3)), ('tooth', (27, 3)), ('lee', (27, 3)), ('dealt', (26, 3)), ('thee', (26, 3)), ('flesh', (26, 3)), ('sees', (26, 3)), ('upset', (26, 3)), ('key', (26, 3)), ('zeal', (25, 3)), ('agent', (25, 3)), ('dose', (25, 3)), ('pen', (25, 3)), ('weep', (25, 3)), ('suit', (25, 3)), ('en', (25, 3)), ('desert', (25, 3)), ('merit', (24, 3)), ('bees', (24, 3)), ('fleet', (24, 3)), ('root', (24, 3)), ('fed', (24, 3)), ('wise', (24, 3)), ('honest', (23, 3)), ('waste', (23, 3)), ('pas', (23, 3)), ('duct', (22, 3)), ('posts', (22, 3)), ('boot', (22, 3)), ('web', (21, 3)), ('tale', (21, 3)), ('fish', (21, 3)), ('ate', (21, 3)), ('hit', (21, 3)), ('etc', (21, 3)), ('seats', (21, 3)), ('heel', (21, 3)), ('pose', (21, 3)), ('vested', (21, 3)), ('halt', (20, 3)), ('wash', (20, 3)), ('tiny', (20, 3)), ('tells', (20, 3)), ('protest', (19, 3)), ('port', (19, 3)), ('sweat', (19, 3)), ('fort', (19, 3)), ('push', (19, 3)), ('bass', (18, 3)), ('bend', (18, 3)), ('tore', (18, 3)), ('ex', (18, 3)), ('semi', (18, 3)), ('exit', (18, 3)), ('geese', (18, 3)), ('waist', (17, 3)), ('tales', (17, 3)), ('den', (17, 3)), ('scent', (17, 3)), ('guess', (17, 3)), ('heap', (17, 3)), ('arts', (17, 3)), ('petty', (17, 3)), ('erect', (17, 3)), ('lowest', (17, 3)), ('dies', (17, 3)), ('je', (17, 3)), ('dusty', (16, 3)), ('serf', (16, 3)), ('ages', (16, 3)), ('gas', (16, 3)), ('knot', (16, 3)), ('tips', (16, 3)), ('veil', (16, 3)), ('wit', (15, 3)), ('fees', (15, 3)), ('oats', (15, 3)), ('writ', (15, 3)), ('bee', (15, 3)), ('seal', (15, 3)), ('pitt', (15, 3)), ('seed', (15, 3)), ('herd', (15, 3)), ('tune', (15, 3)), ('gems', (15, 3)), ('heed', (15, 3)), ('rep', (15, 3)), ('meal', (15, 3)), ('pit', (15, 3)), ('tie', (15, 3)), ('wasn', (14, 3)), ('knit', (14, 3)), ('dirt', (14, 3)), ('fits', (14, 3)), ('egg', (14, 3)), ('mustn', (14, 3)), ('penn', (14, 3)), ('quit', (14, 3)), ('ross', (14, 3)), ('toil', (14, 3)), ('ed', (14, 3)), ('pea', (14, 3)), ('gut', (14, 3)), ('dish', (13, 3)), ('tubes', (13, 3)), ('modest', (13, 3)), ('rat', (13, 3)), ('lint', (13, 3)), ('keys', (13, 3)), ('alert', (13, 3)), ('hint', (13, 3)), ('diet', (13, 3)), ('beds', (13, 3)), ('cell', (12, 3)), ('elect', (12, 3)), ('plot', (12, 3)), ('deity', (12, 3)), ('leash', (12, 3)), ('steep', (12, 3)), ('nasty', (12, 3)), ('gout', (12, 3)), ('cot', (12, 3)), ('tour', (12, 3)), ('tread', (12, 3)), ('lean', (12, 3)), ('trip', (12, 3)), ('ne', (12, 3)), ('lens', (12, 3)), ('mask', (12, 3)), ('lend', (11, 3)), ('avert', (11, 3)), ('fete', (11, 3)), ('hemp', (11, 3)), ('resort', (11, 3)), ('wens', (11, 3)), ('finest', (11, 3)), ('jesus', (11, 3)), ('cash', (11, 3)), ('sex', (11, 3)), ('rein', (11, 3)), ('boat', (11, 3)), ('cents', (11, 3)), ('fuss', (11, 3)), ('ce', (11, 3)), ('gift', (11, 3)), ('paste', (11, 3)), ('rely', (11, 3)), ('riot', (11, 3)), ('hats', (10, 3)), ('ney', (10, 3)), ('dew', (10, 3)), ('crust', (10, 3)), ('unit', (10, 3)), ('heir', (10, 3)), ('cat', (10, 3)), ('bites', (10, 3)), ('tap', (10, 3)), ('thud', (10, 3)), ('wont', (10, 3)), ('bless', (10, 3)), ('dusk', (10, 3)), ('feed', (10, 3)), ('deny', (10, 3)), ('nut', (10, 3)), ('foes', (10, 3)), ('era', (10, 3)), ('deaf', (10, 3)), ('disk', (10, 3)), ('lasts', (10, 3)), ('beef', (10, 3)), ('misty', (10, 3)), ('hey', (10, 3)), ('fists', (9, 3)), ('knelt', (9, 3)), ('hasn', (9, 3)), ('raft', (9, 3)), ('dash', (9, 3)), ('greet', (9, 3)), ('crept', (9, 3)), ('veto', (9, 3)), ('dwelt', (9, 3)), ('lists', (9, 3)), ('sera', (9, 3)), ('tract', (9, 3)), ('ze', (9, 3)), ('otis', (9, 3)), ('sites', (9, 3)), ('dates', (9, 3)), ('tons', (9, 3)), ('beer', (9, 3)), ('uses', (9, 3)), ('jews', (9, 3)), ('tabes', (9, 3)), ('chat', (9, 3)), ('vs', (9, 3)), ('unrest', (8, 3)), ('rents', (8, 3)), ('trend', (8, 3)), ('meek', (8, 3)), ('eats', (8, 3)), ('theme', (8, 3)), ('dem', (8, 3)), ('bush', (8, 3)), ('roast', (8, 3)), ('tray', (8, 3)), ('bits', (8, 3)), ('exert', (8, 3)), ('fe', (8, 3)), ('vewy', (8, 3)), ('lots', (8, 3)), ('tula', (8, 3)), ('huts', (8, 3)), ('lash', (8, 3)), ('pot', (8, 3)), ('verse', (8, 3)), ('tip', (8, 3)), ('ye', (8, 3)), ('user', (8, 3)), ('costs', (8, 3)), ('stein', (8, 3)), ('gait', (8, 3)), ('gilt', (8, 3)), ('pew', (7, 3)), ('ti', (7, 3)), ('blast', (7, 3)), ('tver', (7, 3)), ('trait', (7, 3)), ('tar', (7, 3)), ('hasty', (7, 3)), ('septa', (7, 3)), ('toy', (7, 3)), ('ton', (7, 3)), ('tool', (7, 3)), ('butt', (7, 3)), ('wits', (7, 3)), ('tufts', (7, 3)), ('mot', (7, 3)), ('tubs', (7, 3)), ('tendo', (7, 3)), ('germ', (7, 3)), ('leaf', (7, 3)), ('tin', (7, 3)), ('rash', (7, 3)), ('ghost', (7, 3)), ('yell', (7, 3)), ('disc', (6, 3)), ('oldest', (6, 3)), ('tweed', (6, 3)), ('ta', (6, 3)), ('jerk', (6, 3)), ('beasts', (6, 3)), ('cette', (6, 3)), ('ein', (6, 3)), ('slit', (6, 3)), ('nuts', (6, 3)), ('pint', (6, 3)), ('bestow', (6, 3)), ('omit', (6, 3)), ('zat', (6, 3)), ('sero', (6, 3)), ('moss', (6, 3)), ('steal', (6, 3)), ('testing', (6, 3)), ('wisp', (6, 3)), ('debs', (6, 3)), ('leo', (6, 3)), ('tenor', (6, 3)), ('sits', (6, 3)), ('resect', (6, 3)), ('hosts', (6, 3)), ('hush', (6, 3)), ('hen', (6, 3)), ('vaso', (6, 3)), ('deem', (6, 3)), ('fuse', (6, 3)), ('item', (5, 3)), ('ken', (5, 3)), ('neo', (5, 3)), ('axes', (5, 3)), ('herb', (5, 3)), ('genet', (5, 3)), ('bolt', (5, 3)), ('er', (5, 3)), ('ash', (5, 3)), ('der', (5, 3)), ('wart', (5, 3)), ('erb', (5, 3)), ('sewn', (5, 3)), ('weed', (5, 3)), ('tier', (5, 3)), ('pens', (5, 3)), ('tory', (5, 3)), ('em', (5, 3)), ('rev', (5, 3)), ('kit', (5, 3)), ('taint', (5, 3)), ('zero', (5, 3)), ('si', (5, 3)), ('chess', (5, 3)), ('versa', (5, 3)), ('react', (5, 3)), ('tries', (5, 3)), ('pegs', (5, 3)), ('fait', (5, 3)), ('boss', (5, 3)), ('ants', (5, 3)), ('cuts', (5, 3)), ('wisest', (5, 3)), ('widest', (5, 3)), ('stem', (5, 3)), ('jena', (5, 3)), ('trim', (5, 3)), ('deer', (5, 3)), ('tu', (4, 3)), ('leap', (4, 3)), ('rusty', (4, 3)), ('twin', (4, 3)), ('boast', (4, 3)), ('items', (4, 3)), ('pere', (4, 3)), ('chit', (4, 3)), ('levy', (4, 3)), ('sa', (4, 3)), ('jem', (4, 3)), ('bean', (4, 3)), ('hell', (4, 3)), ('digest', (4, 3)), ('hem', (4, 3)), ('edit', (4, 3)), ('veldt', (4, 3)), ('defy', (4, 3)), ('inert', (4, 3)), ('fret', (4, 3)), ('isa', (4, 3)), ('gear', (4, 3)), ('tom', (4, 3)), ('mates', (4, 3)), ('faut', (4, 3)), ('reed', (4, 3)), ('cheat', (4, 3)), ('purest', (4, 3)), ('sait', (4, 3)), ('adept', (4, 3)), ('tub', (4, 3)), ('tug', (4, 3)), ('pits', (4, 3)), ('len', (4, 3)), ('steer', (4, 3)), ('etre', (4, 3)), ('rete', (4, 3)), ('poet', (4, 3)), ('jesuit', (4, 3)), ('tidy', (4, 3)), ('ruts', (4, 3)), ('overt', (4, 3)), ('peg', (4, 3)), ('peer', (4, 3)), ('peru', (4, 3)), ('ut', (4, 3)), ('ruse', (4, 3)), ('apes', (4, 3)), ('petit', (4, 3)), ('beats', (4, 3)), ('merest', (4, 3)), ('smelt', (4, 3)), ('peak', (4, 3)), ('hilt', (3, 3)), ('teres', (3, 3)), ('jew', (3, 3)), ('tempest', (3, 3)), ('ant', (3, 3)), ('mews', (3, 3)), ('genu', (3, 3)), ('os', (3, 3)), ('ices', (3, 3)), ('twy', (3, 3)), ('dewy', (3, 3)), ('epi', (3, 3)), ('twos', (3, 3)), ('rapt', (3, 3)), ('despot', (3, 3)), ('goat', (3, 3)), ('mat', (3, 3)), ('herr', (3, 3)), ('deck', (3, 3)), ('soot', (3, 3)), ('girt', (3, 3)), ('twas', (3, 3)), ('whit', (3, 3)), ('testify', (3, 3)), ('sash', (3, 3)), ('temps', (3, 3)), ('keith', (3, 3)), ('tack', (3, 3)), ('turk', (3, 3)), ('dint', (3, 3)), ('los', (3, 3)), ('pots', (3, 3)), ('lease', (3, 3)), ('beck', (3, 3)), ('watt', (3, 3)), ('se', (3, 3)), ('trod', (3, 3)), ('troy', (3, 3)), ('font', (3, 3)), ('tire', (3, 3)), ('mend', (3, 3)), ('resent', (3, 3)), ('beam', (3, 3)), ('goot', (3, 3)), ('bead', (3, 3)), ('gem', (3, 3)), ('sects', (3, 3)), ('irs', (3, 3)), ('hiss', (3, 3)), ('weal', (3, 3)), ('tacit', (3, 3)), ('invest', (3, 3)), ('dues', (3, 3)), ('vex', (3, 3)), ('gasp', (3, 3)), ('vena', (3, 3)), ('turf', (3, 3)), ('loot', (3, 3)), ('pat', (3, 3)), ('meets', (3, 3)), ('tops', (3, 3)), ('costo', (3, 3)), ('meuse', (3, 3)), ('vilest', (3, 3)), ('poets', (3, 3)), ('eau', (3, 3)), ('pear', (3, 3)), ('jested', (3, 3)), ('puts', (3, 3)), ('tomb', (3, 3)), ('fera', (3, 3)), ('relit', (3, 3)), ('emit', (3, 3)), ('dites', (3, 3)), ('wee', (3, 3)), ('didst', (3, 3)), ('rift', (3, 3)), ('depot', (3, 3)), ('tic', (3, 3)), ('surest', (3, 3)), ('peas', (3, 3)), ('nasi', (3, 3)), ('naso', (3, 3)), ('bait', (2, 3)), ('verb', (2, 3)), ('err', (2, 3)), ('wurst', (2, 3)), ('vell', (2, 3)), ('rats', (2, 3)), ('ned', (2, 3)), ('nez', (2, 3)), ('boost', (2, 3)), ('cela', (2, 3)), ('tags', (2, 3)), ('testily', (2, 3)), ('peep', (2, 3)), ('mort', (2, 3)), ('neve', (2, 3)), ('dot', (2, 3)), ('vot', (2, 3)), ('berth', (2, 3)), ('mots', (2, 3)), ('dec', (2, 3)), ('def', (2, 3)), ('pont', (2, 3)), ('tag', (2, 3)), ('iso', (2, 3)), ('gash', (2, 3)), ('ft', (2, 3)), ('sont', (2, 3)), ('peur', (2, 3)), ('colt', (2, 3)), ('ebb', (2, 3)), ('wilt', (2, 3)), ('scot', (2, 3)), ('lech', (2, 3)), ('bout', (2, 3)), ('dat', (2, 3)), ('debut', (2, 3)), ('tra', (2, 3)), ('yeah', (2, 3)), ('mats', (2, 3)), ('mead', (2, 3)), ('welsh', (2, 3)), ('ass', (2, 3)), ('ms', (2, 3)), ('dont', (2, 3)), ('iss', (2, 3)), ('cats', (2, 3)), ('thaw', (2, 3)), ('weld', (2, 3)), ('heh', (2, 3)), ('entr', (2, 3)), ('rout', (2, 3)), ('gavest', (2, 3)), ('masts', (2, 3)), ('ke', (2, 3)), ('yeas', (2, 3)), ('kits', (2, 3)), ('pe', (2, 3)), ('beri', (2, 3)), ('tape', (2, 3)), ('beau', (2, 3)), ('beak', (2, 3)), ('bat', (2, 3)), ('belts', (2, 3)), ('stead', (2, 3)), ('barest', (2, 3)), ('jot', (2, 3)), ('keg', (2, 3)), ('peck', (2, 3)), ('tuck', (2, 3)), ('til', (2, 3)), ('grs', (2, 3)), ('esch', (2, 3)), ('spit', (2, 3)), ('stems', (2, 3)), ('scut', (2, 3)), ('tank', (2, 3)), ('tapes', (2, 3)), ('stint', (2, 3)), ('dart', (2, 3)), ('leapt', (2, 3)), ('oft', (2, 3)), ('ely', (2, 3)), ('hewn', (2, 3)), ('reap', (2, 3)), ('med', (2, 3)), ('safest', (2, 3)), ('resin', (2, 3)), ('texan', (2, 3)), ('meats', (2, 3)), ('mash', (2, 3)), ('peu', (2, 3)), ('toys', (2, 3)), ('beget', (2, 3)), ('trout', (2, 3)), ('slut', (2, 3)), ('fetes', (2, 3)), ('rut', (2, 3)), ('oe', (2, 3)), ('faust', (2, 3)), ('ores', (2, 3)), ('vise', (2, 3)), ('tafa', (2, 3)), ('ver', (2, 3)), ('spat', (2, 3)), ('asset', (2, 3)), ('toi', (2, 3)), ('vere', (2, 3)), ('curt', (2, 3)), ('blot', (2, 3)), ('vista', (2, 3)), ('weir', (2, 3)), ('peal', (2, 3)), ('yelp', (2, 3)), ('heath', (2, 3)), ('caste', (2, 3)), ('gene', (1, 3)), ('veal', (1, 3)), ('melts', (1, 3)), ('steve', (1, 3)), ('etait', (1, 3)), ('rot', (1, 3)), ('tz', (1, 3)), ('ty', (1, 3)), ('tr', (1, 3)), ('weyl', (1, 3)), ('sens', (1, 3)), ('dey', (1, 3)), ('fas', (1, 3)), ('rec', (1, 3)), ('voit', (1, 3)), ('yeux', (1, 3)), ('vos', (1, 3)), ('lisa', (1, 3)), ('whist', (1, 3)), ('cms', (1, 3)), ('hesse', (1, 3)), ('dis', (1, 3)), ('mayest', (1, 3)), ('gase', (1, 3)), ('etats', (1, 3)), ('cosy', (1, 3)), ('trop', (1, 3)), ('ere', (1, 3)), ('veux', (1, 3)), ('musty', (1, 3)), ('melk', (1, 3)), ('slot', (1, 3)), ('meg', (1, 3)), ('ernest', (1, 3)), ('ghent', (1, 3)), ('hasp', (1, 3)), ('jose', (1, 3)), ('menu', (1, 3)), ('twue', (1, 3)), ('gosp', (1, 3)), ('tutti', (1, 3)), ('feu', (1, 3)), ('freest', (1, 3)), ('repast', (1, 3)), ('holt', (1, 3)), ('muse', (1, 3)), ('muss', (1, 3)), ('felts', (1, 3)), ('cult', (1, 3)), ('gweat', (1, 3)), ('malt', (1, 3)), ('mt', (1, 3)), ('del', (1, 3)), ('duets', (1, 3)), ('hits', (1, 3)), ('weib', (1, 3)), ('punt', (1, 3)), ('tile', (1, 3)), ('vasa', (1, 3)), ('vient', (1, 3)), ('strut', (1, 3)), ('sq', (1, 3)), ('las', (1, 3)), ('sg', (1, 3)), ('sd', (1, 3)), ('fattest', (1, 3)), ('metes', (1, 3)), ('eton', (1, 3)), ('dere', (1, 3)), ('helm', (1, 3)), ('wat', (1, 3)), ('cret', (1, 3)), ('feats', (1, 3)), ('bunt', (1, 3)), ('aft', (1, 3)), ('das', (1, 3)), ('leon', (1, 3)), ('reek', (1, 3)), ('leak', (1, 3)), ('zen', (1, 3)), ('woes', (1, 3)), ('busts', (1, 3)), ('reef', (1, 3)), ('mete', (1, 3)), ('ras', (1, 3)), ('gee', (1, 3)), ('matt', (1, 3)), ('debit', (1, 3)), ('asp', (1, 3)), ('trite', (1, 3)), ('bask', (1, 3)), ('gott', (1, 3)), ('fiat', (1, 3)), ('grist', (1, 3)), ('recast', (1, 3)), ('lout', (1, 3)), ('gusto', (1, 3)), ('att', (1, 3)), ('ism', (1, 3)), ('sew', (1, 3)), ('sn', (1, 3)), ('ont', (1, 3)), ('tate', (1, 3)), ('tow', (1, 3)), ('hearst', (1, 3)), ('meme', (1, 3)), ('lisp', (1, 3)), ('zeus', (1, 3)), ('lush', (1, 3)), ('vestry', (1, 3)), ('tires', (1, 3)), ('recto', (1, 3)), ('sta', (1, 3)), ('steed', (1, 3)), ('vat', (1, 3)), ('sty', (1, 3)), ('telly', (1, 3)), ('wen', (1, 3)), ('mint', (1, 3)), ('utero', (1, 3)), ('sein', (1, 3)), ('desist', (1, 3)), ('wed', (1, 3)), ('deux', (1, 3)), ('lis', (1, 3)), ('takh', (1, 3)), ('elm', (1, 3)), ('plat', (1, 3)), ('beta', (1, 3)), ('heah', (1, 3)), ('vish', (1, 3)), ('beth', (1, 3)), ('jean', (1, 3)), ('taxi', (1, 3)), ('nee', (1, 3)), ('woot', (1, 3)), ('jolt', (1, 3)), ('oct', (1, 3)), ('brat', (1, 3)), ('tyre', (1, 3)), ('owes', (1, 3)), ('ben', (1, 3)), ('lea', (1, 3)), ('reid', (1, 3)), ('gush', (1, 3)), ('tunes', (1, 3)), ('bart', (1, 3)), ('sez', (1, 3)), ('wead', (1, 3)), ('hoes', (1, 3)), ('sexe', (1, 3)), ('hise', (1, 3)), ('yer', (1, 3)), ('wirt', (1, 3)), ('durst', (1, 3)), ('diese', (1, 3)), ('fez', (1, 3)), ('ety', (1, 3)), ('quests', (1, 3)), ('fester', (1, 3)), ('rosa', (1, 3)), ('walt', (1, 3)), ('eva', (1, 3)), ('tony', (1, 3)), ('teddy', (1, 3)), ('tiens', (1, 3)), ('hottest', (1, 3)), ('dots', (1, 3)), ('obese', (1, 3)), ('vase', (1, 3)), ('tiers', (1, 3)), ('gen', (1, 3)), ('ego', (1, 3)), ('nates', (1, 3)), ('tim', (1, 3)), ('cusp', (1, 3)), ('theah', (1, 3)), ('mott', (1, 3)), ('twig', (1, 3)), ('arat', (1, 3)), ('elets', (1, 3)), ('jeff', (1, 3)), ('lusty', (1, 3)), ('costa', (1, 3)), ('tyne', (1, 3)), ('kempt', (1, 3)), ('pott', (1, 3)), ('levi', (1, 3)), ('newest', (1, 3)), ('yea', (1, 3)), ('ablest', (1, 3)), ('yep', (1, 3)), ('hetty', (1, 3)), ('tante', (1, 3)), ('trestle', (1, 3)), ('zere', (1, 3)), ('verso', (1, 3)), ('opt', (1, 3)), ('grit', (1, 3)), ('taunt', (1, 3)), ('fats', (1, 3)), ('cleft', (1, 3)), ('tick', (1, 3)), ('pies', (1, 3)), ('mein', (1, 3)), ('vis', (1, 3)), ('vestas', (1, 3)), ('divest', (1, 3)), ('canst', (1, 3)), ('dean', (1, 3)), ('chests', (1, 3)), ('mele', (1, 3)), ('sots', (1, 3)), ('hew', (1, 3)), ('pasty', (1, 3)), ('stew', (1, 3)), ('feb', (1, 3)), ('wesel', (1, 3)), ('zis', (1, 3)), ('felo', (1, 3)), ('vult', (1, 3)), ('justo', (1, 3)), ('nos', (1, 3)), ('ingest', (1, 3)), ('trata', (1, 3)), ('obs', (1, 3)), ('el', (1, 3)), ('deah', (1, 3)), ('sot', (1, 3)), ('git', (1, 3)), ('tame', (1, 3)), ('fates', (1, 3)), ('dews', (1, 3)), ('vas', (1, 3)), ('eg', (1, 3)), ('tints', (1, 3)), ('bus', (1, 3)), ('mists', (1, 3)), ('atheist', (1, 3)), ('tiniest', (1, 3)), ('shit', (1, 3)), ('ev', (1, 3)), ('cos', (1, 3)), ('titi', (1, 3)), ('vesna', (1, 3)), ('tides', (1, 3)), ('fewest', (1, 3)), ('nicest', (1, 3)), ('felty', (1, 3)), ('tepid', (1, 3)), ('casts', (1, 3)), ('wetu', (1, 3))]

In [57]:
test2 = [(i[0], i[1][1]) for i in test]
len(test2)

1124

In [77]:
print len([i[0] for i in test2 if i[1]==1])
print [i[0] for i in test2 if i[1]==1]

31
['in', 'it', 'his', 'as', 'i', 's', 'if', 'its', 'us', 'ii', 'iv', 'isn', 'ix', 'il', 'vs', 'si', 'tis', 'isa', 'es', 'irs', 'os', 'ms', 'iss', 'iso', 'dis', 'vis', 'zis', 'ist', 'ism', 'lis', 'id']


<div class="alert alert-info">
<p><b>For testing:</b></p>
<p>
Provide text file to correct, and give all best word suggestions (word level & context level).
</p>
</div>

In [ ]:
%%time
correct_document_context("testdata/test.txt", \
                         dictionary, longest_word_length, start_prob, default_start_prob, \
                         transition_prob, default_transition_prob)

In [ ]:
%%time
correct_document_context("testdata/OCRsample.txt", \
                         dictionary, longest_word_length, start_prob, default_start_prob, \
                         transition_prob, default_transition_prob)

In [ ]:
%%time
correct_document_context("testdata/tiny.txt", \
                         dictionary, longest_word_length, start_prob, default_start_prob, \
                         transition_prob, default_transition_prob)